In [ ]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from datetime import datetime
from transformers import pipeline
from fastapi import FastAPI
from bson import ObjectId
url = "https://laodong.vn/xa-hoi/chuan-bi-khoi-cong-loat-du-an-giao-thong-lon-1586625.ldo"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
content_div = soup.select_one("div.detail-content, div.article-content")
  # hoặc thử "detail-content", "content-detail"
if content_div:
    article_text = content_div.get_text(strip=True)
    print(article_text)
else:
    article_text = None
    print("Không tìm thấy nội dung bài báo.")
if article_text:
    print("Đã lấy được nội dung.")
else:
    print("Không có nội dung để hiển thị.")
print(article_text)
print(soup.prettify())
client = MongoClient("mongodb://localhost:27017/")
db = client["traffic_news"]
collection = db["articles"]
document = {
    "url": url,
    "title": "Chuẩn bị khởi công loạt dự án giao thông lớn",
    "content": article_text,
    "date": datetime.now()
}
collection.insert_one(document)
summarizer = pipeline("summarization", model="VietAI/vit5-base-vietnews-summarization")
summary = summarizer(article_text, max_length=200, min_length=50, do_sample=False)[0]["summary_text"]
print("Tóm tắt:", summary)
app = FastAPI()
@app.get("/summary")
def get_summary(article_id: str):
    doc = collection.find_one({"_id": ObjectId(article_id)})
    summary = summarizer(doc["content"], max_length=200)[0]["summary_text"]
    return {"title": doc["title"], "summary": summary}
    collection.update_one({"_id": doc["_id"]}, {"$set": {"summary": summary}})

Không tìm thấy nội dung bài báo.
Không có nội dung để hiển thị.
None
<html>
 <body>
  <script>
   document.cookie="D1N=4492fe22b0f8939a6e2cac59d196c3e2"+"; expires=Fri, 31 Dec 2099 23:59:59 GMT; path=/";window.location.reload(true);
  </script>
 </body>
</html>



In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient
from datetime import datetime
from fastapi import FastAPI, HTTPException
from bson import ObjectId
import time

# Khởi tạo model (fallback mode - không cần PyTorch)
print("⚠ Dùng tóm tắt đơn giản (3 câu đầu)")
USE_AI_SUMMARY = False

# Kết nối MongoDB
try:
    client = MongoClient("mongodb://localhost:27017/", serverSelectionTimeoutMS=5000)
    client.server_info()
    db = client["traffic_news"]
    collection = db["articles"]
    print("✓ Đã kết nối MongoDB")
except Exception as e:
    print(f"✗ Lỗi kết nối MongoDB: {e}")
    collection = None

def scrape_article_selenium(url):
    """Lấy nội dung bài báo bằng Selenium"""
    driver = None
    try:
        # Setup Edge headless
        edge_options = webdriver.EdgeOptions()
        edge_options.add_argument("--headless")  # Chạy ngầm, không mở cửa sổ
        edge_options.add_argument("--no-sandbox")
        edge_options.add_argument("--disable-dev-shm-usage")
        edge_options.add_argument("--disable-gpu")
        edge_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
        
        print("Đang khởi động Edge...")
        driver = webdriver.Edge(options=edge_options)
        driver.set_page_load_timeout(30)
        
        print(f"Đang tải trang: {url}")
        driver.get(url)
        
        # Đợi JavaScript load xong (đợi nội dung xuất hiện)
        print("Đang đợi nội dung load...")
        wait = WebDriverWait(driver, 15)
        
        # Thử nhiều selector (Lao Động thường dùng)
        selectors = [
            (By.CSS_SELECTOR, "div.detail-content-body"),
            (By.CSS_SELECTOR, "div.detail__content"),
            (By.CSS_SELECTOR, "article.detail"),
            (By.CSS_SELECTOR, "div.article-content"),
            (By.XPATH, "//div[contains(@class, 'detail') and contains(@class, 'content')]"),
            (By.TAG_NAME, "article")
        ]
        
        content_element = None
        used_selector = None
        
        for by, selector in selectors:
            try:
                content_element = wait.until(
                    EC.presence_of_element_located((by, selector))
                )
                if content_element and len(content_element.text) > 200:
                    used_selector = selector
                    print(f"✓ Tìm thấy nội dung với: {selector}")
                    break
            except:
                continue
        
        if not content_element:
            # Fallback: Đợi 3s rồi lấy toàn bộ body
            print("⚠ Không tìm thấy selector cụ thể, lấy toàn bộ body...")
            time.sleep(3)
            content_element = driver.find_element(By.TAG_NAME, "body")
        
        # Lấy title
        try:
            title_element = driver.find_element(By.CSS_SELECTOR, "h1.detail__title, h1")
            title = title_element.text.strip()
        except:
            title = driver.title
        
        # Lấy nội dung
        article_text = content_element.text.strip()
        
        # Làm sạch (loại bỏ navigation, ads...)
        lines = [line.strip() for line in article_text.split("\n") if line.strip()]
        # Loại bỏ các dòng ngắn (thường là menu, ads)
        lines = [line for line in lines if len(line) > 30]
        article_text = " ".join(lines)
        
        print(f"✓ Đã lấy được {len(article_text)} ký tự")
        
        return title, article_text
        
    except Exception as e:
        print(f"✗ Lỗi Selenium: {e}")
        print(f"   Tip: Đảm bảo Edge đã được cài đặt hoặc cài Edge WebDriver")
        return None, None
    finally:
        if driver:
            driver.quit()
            print("✓ Đã đóng Edge")

def simple_summary(text, num_sentences=3):
    """Tóm tắt đơn giản bằng cách lấy N câu đầu"""
    sentences = text.replace("?", ".").replace("!", ".").split(". ")
    sentences = [s.strip() for s in sentences if len(s.strip()) > 20]
    return ". ".join(sentences[:num_sentences]) + "."

# Scraping
url = "https://laodong.vn/xa-hoi/chuan-bi-khoi-cong-loat-du-an-giao-thong-lon-1586625.ldo"
print(f"\n{'='*60}")
print(f"BẮT ĐẦU SCRAPE")
print(f"{'='*60}\n")

title, article_text = scrape_article_selenium(url)

if article_text and len(article_text) > 100:
    print(f"\n✓ Tiêu đề: {title[:100]}...")
    print(f"✓ Nội dung: {len(article_text)} ký tự")
    print(f"\nNội dung đầu tiên:\n{article_text[:300]}...\n")
    
    # Tóm tắt
    summary = simple_summary(article_text, num_sentences=3)
    print(f"{'='*60}")
    print(f"TÓM TẮT:")
    print(f"{'='*60}")
    print(summary)
    print(f"{'='*60}\n")
    
    # Lưu vào MongoDB
    if collection is not None:
        try:
            document = {
                "url": url,
                "title": title,
                "content": article_text,
                "summary": summary,
                "date": datetime.now(),
                "scraped_at": datetime.now()
            }
            result = collection.insert_one(document)
            print(f"✓ Đã lưu vào MongoDB với ID: {result.inserted_id}\n")
        except Exception as e:
            print(f"✗ Lỗi khi lưu MongoDB: {e}\n")
else:
    print("✗ Không lấy được nội dung đủ dài\n")

# FastAPI
app = FastAPI(title="Traffic News API")

@app.get("/")
def root():
    return {
        "message": "Traffic News Summarization API",
        "endpoints": {
            "GET /articles": "Danh sách bài viết",
            "GET /summary/{article_id}": "Tóm tắt bài viết",
            "POST /scrape": "Scrape URL mới (body: {\"url\": \"...\"})"
        }
    }

@app.get("/summary/{article_id}")
def get_summary(article_id: str):
    """Lấy tóm tắt bài viết theo ID"""
    if collection is None:
        raise HTTPException(status_code=503, detail="MongoDB không khả dụng")
    
    try:
        doc = collection.find_one({"_id": ObjectId(article_id)})
        
        if not doc:
            raise HTTPException(status_code=404, detail="Không tìm thấy bài viết")
        
        # Nếu đã có summary
        if "summary" in doc and doc["summary"]:
            return {
                "id": str(doc["_id"]),
                "title": doc["title"],
                "summary": doc["summary"],
                "content_length": len(doc.get("content", "")),
                "cached": True
            }
        
        # Tạo mới
        if not doc.get("content"):
            raise HTTPException(status_code=400, detail="Bài viết không có nội dung")
        
        summary = simple_summary(doc["content"], num_sentences=3)
        
        # Cập nhật vào DB
        collection.update_one(
            {"_id": doc["_id"]}, 
            {"$set": {"summary": summary}}
        )
        
        return {
            "id": str(doc["_id"]),
            "title": doc["title"],
            "summary": summary,
            "content_length": len(doc["content"]),
            "cached": False
        }
        
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Lỗi: {str(e)}")

@app.get("/articles")
def list_articles(limit: int = 10):
    """Liệt kê bài viết"""
    if collection is None:
        raise HTTPException(status_code=503, detail="MongoDB không khả dụng")
    
    docs = collection.find().sort("date", -1).limit(limit)
    articles = []
    for doc in docs:
        articles.append({
            "id": str(doc["_id"]),
            "title": doc["title"],
            "date": doc["date"].isoformat(),
            "url": doc.get("url", ""),
            "has_summary": "summary" in doc
        })
    return {"articles": articles, "count": len(articles)}

@app.post("/scrape")
def scrape_new_article(data: dict):
    """Scrape URL mới"""
    url = data.get("url")
    if not url:
        raise HTTPException(status_code=400, detail="Thiếu URL")
    
    title, content = scrape_article_selenium(url)
    
    if not content:
        raise HTTPException(status_code=400, detail="Không scrape được nội dung")
    
    summary = simple_summary(content, num_sentences=3)
    
    if collection:
        doc = {
            "url": url,
            "title": title,
            "content": content,
            "summary": summary,
            "date": datetime.now()
        }
        result = collection.insert_one(doc)
        return {
            "id": str(result.inserted_id),
            "title": title,
            "summary": summary,
            "content_length": len(content)
        }
    
    return {"title": title, "summary": summary, "content_length": len(content)}

# Chạy: uvicorn filename:app --reload

⚠ Dùng tóm tắt đơn giản (3 câu đầu)
✓ Đã kết nối MongoDB

BẮT ĐẦU SCRAPE

Đang khởi động Edge...
Đang tải trang: https://laodong.vn/xa-hoi/chuan-bi-khoi-cong-loat-du-an-giao-thong-lon-1586625.ldo
Đang đợi nội dung load...
✓ Tìm thấy nội dung với: article
✓ Đã lấy được 4137 ký tự
✓ Đã đóng Edge

✓ Tiêu đề: Chuẩn bị khởi công loạt dự án giao thông lớn...
✓ Nội dung: 4137 ký tự

Nội dung đầu tiên:
Chuẩn bị khởi công loạt dự án giao thông lớn Thứ hai, 06/10/2025 12:30 (GMT+7) TPHCM chuẩn bị khởi công loạt dự án giao thông lớn trị giá hàng chục nghìn tỉ đồng, từ Metro số 2, cao tốc TPHCM - Mộc Bài đến Vành đai 2, kỳ vọng mở rộng cửa ngõ và thúc đẩy phát triển đô thị. Ùn tắc tại cửa ngõ phía Đôn...

TÓM TẮT:
Chuẩn bị khởi công loạt dự án giao thông lớn Thứ hai, 06/10/2025 12:30 (GMT+7) TPHCM chuẩn bị khởi công loạt dự án giao thông lớn trị giá hàng chục nghìn tỉ đồng, từ Metro số 2, cao tốc TPHCM - Mộc Bài đến Vành đai 2, kỳ vọng mở rộng cửa ngõ và thúc đẩy phát triển đô thị. Ùn tắc tại cửa 

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient
from datetime import datetime, timedelta
from fastapi import FastAPI, HTTPException, BackgroundTasks, Query
from bson import ObjectId
from typing import List, Optional, Dict
import time
import re
import hashlib
from collections import Counter
import math
from difflib import SequenceMatcher
import feedparser
from urllib.parse import urljoin, urlparse

# ==================== CẤU HÌNH NÂNG CAO ====================

# Thời gian lọc
START_DATE = datetime(2025, 1, 1)
CURRENT_DATE = datetime.now()

# Từ khóa TP.HCM (để lọc bài viết)
HCMC_KEYWORDS = [
    "tp.hcm", "tp hcm", "hồ chí minh", "ho chi minh", "sài gòn", "saigon",
    "thành phố hồ chí minh", "tphcm", "hcmc"
]

# Từ khóa giao thông TP.HCM
TRAFFIC_KEYWORDS = {
    "projects": ["metro", "tàu điện", "cao tốc", "cầu", "đường vành đai", "brt", 
                 "khởi công", "hoàn thành", "dự án", "đầu tư", "xây dựng"],
    "infrastructure": ["hạ tầng", "giao thông", "đường bộ", "đường sắt", "đường thủy",
                      "sân bay", "cảng", "bến xe", "trạm", "nút giao thông"],
    "issues": ["kẹt xe", "ùn tắc", "tai nạn", "ngập nước", "ổ gà", "hư hỏng"],
    "planning": ["quy hoạch", "kế hoạch", "phê duyệt", "triển khai", "điều chỉnh"],
    "transport": ["xe buýt", "xe bus", "taxi", "grab", "vận tải", "xe máy", "ô tô"]
}

# Cấu hình nguồn tin chi tiết hơn
NEWS_SOURCES = {
    "laodong": {
        "name": "Báo Lao Động",
        "base_url": "https://laodong.vn",
        "rss_feeds": [
            "https://laodong.vn/rss/giao-thong.rss",
            "https://laodong.vn/rss/xa-hoi.rss"
        ],
        "list_urls": [
            "https://laodong.vn/giao-thong/",
            "https://laodong.vn/giao-thong/trang-{page}.htm"  # Pagination
        ],
        "selectors": {
            "article_links": "h3.title-news a, a[href*='/giao-thong/']",
            "title": "h1.detail__title, h1",
            "content": "div.detail-content-body, div.detail__content",
            "date": "span.time-update, time"
        },
        "date_format": "%d/%m/%Y %H:%M"
    },
    "vnexpress": {
        "name": "VnExpress",
        "base_url": "https://vnexpress.net",
        "rss_feeds": ["https://vnexpress.net/rss/giao-thong.rss"],
        "list_urls": [
            "https://vnexpress.net/giao-thong",
            "https://vnexpress.net/giao-thong-p{page}"
        ],
        "selectors": {
            "article_links": "h3.title-news a, h2.title-news a",
            "title": "h1.title-detail",
            "content": "article.fck_detail, div.fck_detail",
            "date": "span.date"
        },
        "date_format": "%d/%m/%Y, %H:%M"
    },
    "tuoitre": {
        "name": "Báo Tuổi Trẻ",
        "base_url": "https://tuoitre.vn",
        "rss_feeds": ["https://tuoitre.vn/rss/giao-thong.rss"],
        "list_urls": [
            "https://tuoitre.vn/giao-thong.htm",
            "https://tuoitre.vn/giao-thong/trang-{page}.htm"
        ],
        "selectors": {
            "article_links": "h3.title-news a, a[href*='/giao-thong/']",
            "title": "h1.detail-title",
            "content": "div.detail-content, div#main-detail-content",
            "date": "div.date-time"
        },
        "date_format": "%d/%m/%Y %H:%M"
    },
    "hochiminhcity": {
        "name": "Sở GTVT TP.HCM",
        "base_url": "https://giaothong.hochiminhcity.gov.vn",
        "list_urls": [
            "https://giaothong.hochiminhcity.gov.vn/render/rendertintuc.aspx?CateID=1460"
        ],
        "selectors": {
            "article_links": "a[href*='detail']",
            "title": "h1, .title-detail, h1.article-title",
            "content": "div.content-detail, div.article-content, article, div#content",
            "date": "span.date, div.date-time"
        },
        "date_format": "%d/%m/%Y"
    },
    "thanhnien": {
        "name": "Báo Thanh Niên",
        "base_url": "https://thanhnien.vn",
        "rss_feeds": ["https://thanhnien.vn/rss/giao-thong.rss"],
        "list_urls": [
            "https://thanhnien.vn/giao-thong/",
            "https://thanhnien.vn/giao-thong/trang-{page}.html"
        ],
        "selectors": {
            "article_links": "h2.story__heading a, a.story__link",
            "title": "h1.detail-title",
            "content": "div.detail-content, #contentdetail",
            "date": "time, span.time"
        },
        "date_format": "%d/%m/%Y %H:%M"
    }
}

# ==================== KẾT NỐI DATABASE ====================

try:
    client = MongoClient("mongodb://localhost:27017/", serverSelectionTimeoutMS=5000)
    client.server_info()
    db = client["hcmc_traffic_intelligence"]
    articles_col = db["articles"]
    categories_col = db["categories"]
    summaries_col = db["summaries"]
    
    # Index nâng cao
    articles_col.create_index([("url", 1)], unique=True)
    articles_col.create_index([("published_date", -1)])
    articles_col.create_index([("source", 1), ("published_date", -1)])
    articles_col.create_index([("category", 1)])
    articles_col.create_index([("is_hcmc", 1)])
    articles_col.create_index([("content_hash", 1)])
    articles_col.create_index([
        ("title", "text"), 
        ("content", "text"), 
        ("summary", "text")
    ])  # Full-text search
    
    print("✓ Đã kết nối MongoDB với full-text search")
except Exception as e:
    print(f"✗ Lỗi MongoDB: {e}")
    articles_col = None

# ==================== THUẬT TOÁN TÓM TẮT NÂNG CAO ====================

class AdvancedSummarizer:
    """Thuật toán tóm tắt sử dụng TextRank + TF-IDF"""
    
    @staticmethod
    def preprocess_text(text: str) -> List[str]:
        """Tách câu và làm sạch"""
        # Tách câu theo dấu câu
        sentences = re.split(r'[.!?]\s+', text)
        
        # Làm sạch
        cleaned = []
        for sent in sentences:
            sent = sent.strip()
            # Loại bỏ câu quá ngắn hoặc quá dài
            if 20 < len(sent) < 300:
                cleaned.append(sent)
        
        return cleaned
    
    @staticmethod
    def calculate_word_freq(sentences: List[str]) -> Dict[str, float]:
        """Tính tần suất từ (TF)"""
        words = []
        
        # Stopwords tiếng Việt cơ bản
        stopwords = {
            'và', 'của', 'có', 'được', 'là', 'trong', 'với', 'cho', 'các', 
            'một', 'này', 'đã', 'từ', 'những', 'để', 'người', 'không', 'như',
            'về', 'theo', 'năm', 'tại', 'đến', 'khi', 'ngày', 'trên', 'sau',
            'vào', 'thì', 'sẽ', 'ra', 'đang', 'nên', 'bị', 'hay', 'nhưng'
        }
        
        for sent in sentences:
            # Tách từ đơn giản (có thể dùng pyvi để tách từ tốt hơn)
            tokens = re.findall(r'\w+', sent.lower())
            words.extend([w for w in tokens if w not in stopwords and len(w) > 2])
        
        # Tính tần suất
        word_count = Counter(words)
        max_freq = max(word_count.values()) if word_count else 1
        
        return {word: count/max_freq for word, count in word_count.items()}
    
    @staticmethod
    def sentence_similarity(sent1: str, sent2: str, word_freq: Dict[str, float]) -> float:
        """Tính độ tương đồng giữa 2 câu"""
        words1 = set(re.findall(r'\w+', sent1.lower()))
        words2 = set(re.findall(r'\w+', sent2.lower()))
        
        if not words1 or not words2:
            return 0.0
        
        # Tính số từ chung có trọng số
        common_words = words1.intersection(words2)
        score = sum(word_freq.get(word, 0) for word in common_words)
        
        # Normalize
        return score / (math.log(len(words1)) + math.log(len(words2)))
    
    @staticmethod
    def textrank(sentences: List[str], word_freq: Dict[str, float], top_n: int = 3) -> List[str]:
        """TextRank algorithm để chọn câu quan trọng"""
        n = len(sentences)
        if n == 0:
            return []
        
        # Build similarity matrix
        similarity_matrix = [[0.0] * n for _ in range(n)]
        
        for i in range(n):
            for j in range(n):
                if i != j:
                    similarity_matrix[i][j] = AdvancedSummarizer.sentence_similarity(
                        sentences[i], sentences[j], word_freq
                    )
        
        # PageRank algorithm
        scores = [1.0] * n
        damping = 0.85
        
        for _ in range(10):  # 10 iterations
            new_scores = [0.0] * n
            
            for i in range(n):
                rank_sum = sum(
                    similarity_matrix[j][i] * scores[j] / 
                    (sum(similarity_matrix[j]) + 1e-8)
                    for j in range(n) if i != j
                )
                new_scores[i] = (1 - damping) + damping * rank_sum
            
            scores = new_scores
        
        # Chọn top câu và giữ thứ tự xuất hiện
        ranked = [(score, idx, sent) for idx, (score, sent) in enumerate(zip(scores, sentences))]
        ranked.sort(reverse=True)
        
        top_sentences = sorted(ranked[:top_n], key=lambda x: x[1])
        
        return [sent for _, _, sent in top_sentences]
    
    @staticmethod
    def summarize(text: str, num_sentences: int = 3, method: str = "textrank") -> str:
        """Tóm tắt văn bản"""
        sentences = AdvancedSummarizer.preprocess_text(text)
        
        if len(sentences) <= num_sentences:
            return ". ".join(sentences) + "."
        
        if method == "textrank":
            word_freq = AdvancedSummarizer.calculate_word_freq(sentences)
            summary_sents = AdvancedSummarizer.textrank(sentences, word_freq, num_sentences)
        else:
            # Fallback: lấy câu đầu
            summary_sents = sentences[:num_sentences]
        
        return ". ".join(summary_sents) + "."

# ==================== PHÂN LOẠI TỰ ĐỘNG ====================

def categorize_article(title: str, content: str) -> str:
    """Phân loại bài viết theo nội dung"""
    text = (title + " " + content).lower()
    
    scores = {}
    for category, keywords in TRAFFIC_KEYWORDS.items():
        score = sum(1 for kw in keywords if kw in text)
        scores[category] = score
    
    if max(scores.values()) == 0:
        return "general"
    
    return max(scores, key=scores.get)

def is_hcmc_related(title: str, content: str) -> bool:
    """Kiểm tra có liên quan đến TP.HCM không"""
    text = (title + " " + content).lower()
    return any(kw in text for kw in HCMC_KEYWORDS)

# ==================== SCRAPING NÂNG CAO ====================

def init_driver():
    """Khởi tạo Edge driver"""
    edge_options = webdriver.EdgeOptions()
    edge_options.add_argument("--headless")
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--disable-gpu")
    edge_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
    
    driver = webdriver.Edge(options=edge_options)
    driver.set_page_load_timeout(30)
    return driver

def parse_date(date_str: str, date_format: str) -> Optional[datetime]:
    """Parse ngày tháng từ nhiều format"""
    if not date_str:
        return None
    
    try:
        # Thử format được cung cấp
        return datetime.strptime(date_str.strip(), date_format)
    except:
        pass
    
    # Thử các format phổ biến
    formats = [
        "%d/%m/%Y %H:%M",
        "%d/%m/%Y",
        "%d-%m-%Y",
        "%Y-%m-%d",
        "%d.%m.%Y"
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(date_str.strip(), fmt)
        except:
            continue
    
    return None

def scrape_from_rss(source_key: str) -> List[str]:
    """Lấy URLs từ RSS feed"""
    source = NEWS_SOURCES[source_key]
    urls = []
    
    if "rss_feeds" not in source:
        return []
    
    for rss_url in source["rss_feeds"]:
        try:
            print(f"  📡 Đang đọc RSS: {rss_url}")
            feed = feedparser.parse(rss_url)
            
            for entry in feed.entries[:20]:  # Lấy 20 bài mới nhất
                url = entry.get("link", "")
                pub_date = entry.get("published_parsed", None)
                
                # Kiểm tra thời gian
                if pub_date:
                    pub_datetime = datetime(*pub_date[:6])
                    if pub_datetime >= START_DATE:
                        urls.append(url)
                else:
                    urls.append(url)  # Không có ngày thì lấy luôn
            
            print(f"  ✓ Tìm thấy {len(urls)} URLs từ RSS")
        except Exception as e:
            print(f"  ✗ Lỗi đọc RSS: {e}")
    
    return urls

def scrape_from_list_page(source_key: str, max_pages: int = 3) -> List[str]:
    """Lấy URLs từ trang danh sách với pagination"""
    source = NEWS_SOURCES[source_key]
    driver = None
    all_urls = set()
    
    try:
        driver = init_driver()
        
        for page in range(1, max_pages + 1):
            # Lấy URL của trang
            if len(source["list_urls"]) > 1 and page > 1:
                url = source["list_urls"][1].format(page=page)
            else:
                url = source["list_urls"][0]
                if page > 1:
                    break  # Không có pagination
            
            print(f"  📄 Trang {page}: {url[:60]}...")
            driver.get(url)
            time.sleep(3)
            
            # Lấy links
            links = driver.find_elements(By.CSS_SELECTOR, source["selectors"]["article_links"])
            
            page_urls = 0
            for link in links:
                href = link.get_attribute("href")
                if href and href.startswith("http"):
                    all_urls.add(href)
                    page_urls += 1
            
            print(f"  ✓ Tìm thấy {page_urls} URLs")
            
            if page_urls == 0:
                break  # Không còn bài mới
        
        return list(all_urls)
        
    except Exception as e:
        print(f"  ✗ Lỗi scrape list: {e}")
        return list(all_urls)
    finally:
        if driver:
            driver.quit()

def scrape_article_advanced(url: str, source_key: str) -> Optional[Dict]:
    """Scrape 1 bài viết với metadata đầy đủ"""
    source = NEWS_SOURCES[source_key]
    driver = None
    
    try:
        driver = init_driver()
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        
        # Lấy title
        try:
            title_elem = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, source["selectors"]["title"]))
            )
            title = title_elem.text.strip()
        except:
            title = driver.title
        
        # Lấy content
        try:
            content_elem = driver.find_element(By.CSS_SELECTOR, source["selectors"]["content"])
            content = content_elem.text.strip()
        except:
            content = driver.find_element(By.TAG_NAME, "body").text.strip()
        
        # Lấy ngày đăng
        pub_date = None
        if "date" in source["selectors"]:
            try:
                date_elem = driver.find_element(By.CSS_SELECTOR, source["selectors"]["date"])
                date_str = date_elem.text.strip()
                pub_date = parse_date(date_str, source.get("date_format", "%d/%m/%Y"))
            except:
                pass
        
        if pub_date is None:
            pub_date = datetime.now()
        
        # Kiểm tra thời gian
        if pub_date < START_DATE:
            print(f"  ⏭️  Bài cũ ({pub_date.date()}), bỏ qua")
            return None
        
        # Làm sạch content
        lines = [l.strip() for l in content.split("\n") if len(l.strip()) > 30]
        content = " ".join(lines)
        
        # Kiểm tra TP.HCM
        is_hcmc = is_hcmc_related(title, content)
        if not is_hcmc:
            print(f"  ⏭️  Không liên quan TP.HCM")
            return None
        
        # Phân loại
        category = categorize_article(title, content)
        
        # Tóm tắt nâng cao
        summary = AdvancedSummarizer.summarize(content, num_sentences=3, method="textrank")
        
        return {
            "url": url,
            "source": source_key,
            "source_name": source["name"],
            "title": title,
            "content": content,
            "summary": summary,
            "category": category,
            "is_hcmc": is_hcmc,
            "content_hash": hashlib.md5(content.encode()).hexdigest(),
            "published_date": pub_date,
            "scraped_at": datetime.now()
        }
        
    except Exception as e:
        print(f"  ✗ Lỗi scrape: {e}")
        return None
    finally:
        if driver:
            driver.quit()

# ==================== SCRAPE CAMPAIGN ====================

def scrape_source_complete(source_key: str, max_articles: int = 50):
    """Scrape hoàn chỉnh 1 nguồn (RSS + List pages + Date filter)"""
    source = NEWS_SOURCES[source_key]
    
    print(f"\n{'='*60}")
    print(f"🔍 BẮT ĐẦU: {source['name']}")
    print(f"{'='*60}")
    
    # Bước 1: Lấy URLs từ RSS
    print(f"\n📡 Bước 1: Lấy URLs từ RSS...")
    rss_urls = scrape_from_rss(source_key)
    
    # Bước 2: Lấy URLs từ list pages
    print(f"\n📄 Bước 2: Lấy URLs từ trang danh sách...")
    list_urls = scrape_from_list_page(source_key, max_pages=5)
    
    # Gộp và loại trùng
    all_urls = list(set(rss_urls + list_urls))
    print(f"\n📊 Tổng cộng: {len(all_urls)} URLs duy nhất")
    
    # Bước 3: Scrape từng bài
    print(f"\n📰 Bước 3: Scrape chi tiết...")
    
    stats = {
        "total_urls": len(all_urls),
        "scraped": 0,
        "saved": 0,
        "duplicates": 0,
        "not_hcmc": 0,
        "old_articles": 0,
        "errors": 0
    }
    
    for idx, url in enumerate(all_urls[:max_articles], 1):
        print(f"\n[{idx}/{min(len(all_urls), max_articles)}] {url[:70]}...")
        
        # Kiểm tra đã tồn tại
        if articles_col is not None:
            existing = articles_col.find_one({"url": url})
            if existing is not None:
                print(f"  ⏭️  Đã tồn tại trong DB")
                stats["duplicates"] += 1
                continue
        
        # Scrape
        article = scrape_article_advanced(url, source_key)
        
        if article is None:
            stats["errors"] += 1
            continue
        
        stats["scraped"] += 1
        
        # Lưu vào DB
        if articles_col is not None:
            try:
                articles_col.insert_one(article)
                print(f"  ✅ Đã lưu: {article['title'][:50]}...")
                print(f"  📂 Danh mục: {article['category']}")
                print(f"  📅 Ngày: {article['published_date'].date()}")
                stats["saved"] += 1
            except Exception as e:
                print(f"  ✗ Lỗi lưu DB: {e}")
                stats["duplicates"] += 1
        
        time.sleep(2)  # Tránh spam
    
    return stats

def scrape_all_sources_2025(max_per_source: int = 50):
    """Scrape tất cả nguồn với focus vào TP.HCM 2025"""
    print(f"\n{'='*70}")
    print(f"🚀 CHIẾN DỊCH SCRAPING: GIAO THÔNG TP.HCM 2025")
    print(f"   Thời gian: {START_DATE.date()} → {CURRENT_DATE.date()}")
    print(f"{'='*70}\n")
    
    total_stats = {
        "sources": {},
        "grand_total": {
            "urls": 0,
            "scraped": 0,
            "saved": 0,
            "duplicates": 0
        }
    }
    
    for source_key in NEWS_SOURCES.keys():
        stats = scrape_source_complete(source_key, max_per_source)
        total_stats["sources"][source_key] = stats
        
        total_stats["grand_total"]["urls"] += stats["total_urls"]
        total_stats["grand_total"]["scraped"] += stats["scraped"]
        total_stats["grand_total"]["saved"] += stats["saved"]
        total_stats["grand_total"]["duplicates"] += stats["duplicates"]
        
        print(f"\n✅ Hoàn thành {NEWS_SOURCES[source_key]['name']}")
        print(f"   Đã lưu: {stats['saved']} / {stats['scraped']} bài")
    
    return total_stats

# ==================== FASTAPI ====================

app = FastAPI(title="TP.HCM Traffic Intelligence 2025")

@app.get("/")
def root():
    return {
        "name": "TP.HCM Traffic News Intelligence System 2025",
        "period": f"{START_DATE.date()} to {CURRENT_DATE.date()}",
        "sources": len(NEWS_SOURCES),
        "features": [
            "Advanced TextRank summarization",
            "Auto-categorization",
            "HCMC-specific filtering",
            "Full-text search",
            "RSS + Pagination scraping"
        ]
    }

@app.get("/articles")
def list_articles(
    category: Optional[str] = None,
    source: Optional[str] = None,
    from_date: Optional[str] = Query(None, description="YYYY-MM-DD"),
    to_date: Optional[str] = Query(None, description="YYYY-MM-DD"),
    search: Optional[str] = None,
    limit: int = 20
):
    """Danh sách bài viết với filter nâng cao"""
    if articles_col is None:
        raise HTTPException(503, "Database không khả dụng")
    
    query = {"is_hcmc": True}
    
    if category:
        query["category"] = category
    
    if source:
        query["source"] = source
    
    # Date range
    date_query = {}
    if from_date:
        date_query["$gte"] = datetime.strptime(from_date, "%Y-%m-%d")
    if to_date:
        date_query["$lte"] = datetime.strptime(to_date, "%Y-%m-%d")
    if date_query:
        query["published_date"] = date_query
    
    # Full-text search
    if search:
        query["$text"] = {"$search": search}
    
    docs = articles_col.find(query).sort("published_date", -1).limit(limit)
    
    articles = []
    for doc in docs:
        articles.append({
            "id": str(doc["_id"]),
            "title": doc["title"],
            "source": doc["source_name"],
            "category": doc.get("category", "general"),
            "published_date": doc["published_date"].isoformat(),
            "summary": doc.get("summary", "")[:200],
            "url": doc["url"]
        })
    
    return {
        "articles": articles,
        "count": len(articles),
        "filters": {
            "category": category,
            "source": source,
            "from": from_date,
            "to": to_date,
            "search": search
        }
    }

@app.get("/stats")
def get_statistics():
    """Thống kê chi tiết"""
    if articles_col is None:
        raise HTTPException(503, "Database không khả dụng")
    
    total = articles_col.count_documents({"is_hcmc": True})
    
    # By category
    pipeline_category = [
        {"$match": {"is_hcmc": True}},
        {"$group": {"_id": "$category", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
    by_category = {doc["_id"]: doc["count"] for doc in articles_col.aggregate(pipeline_category)}
    
    # By source
    pipeline_source = [
        {"$match": {"is_hcmc": True}},
        {"$group": {"_id": "$source_name", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
    by_source = {doc["_id"]: doc["count"] for doc in articles_col.aggregate(pipeline_source)}
    
    # By month
    pipeline_monthly = [
        {"$match": {"is_hcmc": True, "published_date": {"$gte": START_DATE}}},
        {"$group": {
            "_id": {
                "year": {"$year": "$published_date"},
                "month": {"$month": "$published_date"}
            },
            "count": {"$sum": 1}
        }},
        {"$sort": {"_id.year": 1, "_id.month": 1}}
    ]
    monthly_data = []
    for doc in articles_col.aggregate(pipeline_monthly):
        monthly_data.append({
            "month": f"{doc['_id']['year']}-{doc['_id']['month']:02d}",
            "count": doc["count"]
        })
    
    # Recent articles
    recent_7days = articles_col.count_documents({
        "is_hcmc": True,
        "published_date": {"$gte": datetime.now() - timedelta(days=7)}
    })
    
    return {
        "total_articles": total,
        "articles_last_7days": recent_7days,
        "by_category": by_category,
        "by_source": by_source,
        "monthly_trend": monthly_data,
        "period": {
            "start": START_DATE.isoformat(),
            "end": CURRENT_DATE.isoformat()
        }
    }

@app.get("/articles/{article_id}")
def get_article_detail(article_id: str):
    """Chi tiết bài viết với summary nâng cao"""
    if articles_col is None:
        raise HTTPException(503, "Database không khả dụng")
    
    doc = articles_col.find_one({"_id": ObjectId(article_id)})
    if doc is None:
        raise HTTPException(404, "Không tìm thấy bài viết")
    
    return {
        "id": str(doc["_id"]),
        "title": doc["title"],
        "source": doc["source_name"],
        "category": doc.get("category", "general"),
        "url": doc["url"],
        "content": doc["content"],
        "summary": doc.get("summary", ""),
        "published_date": doc["published_date"].isoformat(),
        "scraped_at": doc["scraped_at"].isoformat(),
        "is_hcmc": doc.get("is_hcmc", False)
    }

@app.get("/categories")
def list_categories():
    """Danh sách các danh mục"""
    return {
        "categories": [
            {"key": "projects", "name": "Dự án", "keywords": TRAFFIC_KEYWORDS["projects"]},
            {"key": "infrastructure", "name": "Hạ tầng", "keywords": TRAFFIC_KEYWORDS["infrastructure"]},
            {"key": "issues", "name": "Vấn đề", "keywords": TRAFFIC_KEYWORDS["issues"]},
            {"key": "planning", "name": "Quy hoạch", "keywords": TRAFFIC_KEYWORDS["planning"]},
            {"key": "transport", "name": "Vận tải", "keywords": TRAFFIC_KEYWORDS["transport"]},
            {"key": "general", "name": "Chung", "keywords": []}
        ]
    }

@app.post("/scrape/all")
def trigger_full_scrape(background_tasks: BackgroundTasks, max_per_source: int = 50):
    """Khởi động scraping toàn bộ (chạy background)"""
    background_tasks.add_task(scrape_all_sources_2025, max_per_source)
    return {
        "status": "started",
        "message": f"Đang scrape tất cả nguồn (tối đa {max_per_source} bài/nguồn)",
        "target_period": f"{START_DATE.date()} to {CURRENT_DATE.date()}"
    }

@app.post("/scrape/{source_key}")
def trigger_source_scrape(source_key: str, max_articles: int = 50):
    """Scrape 1 nguồn cụ thể"""
    if source_key not in NEWS_SOURCES:
        raise HTTPException(404, f"Nguồn '{source_key}' không tồn tại")
    
    stats = scrape_source_complete(source_key, max_articles)
    
    return {
        "source": NEWS_SOURCES[source_key]["name"],
        "stats": stats
    }

@app.get("/search")
def search_articles(
    q: str = Query(..., description="Từ khóa tìm kiếm"),
    limit: int = 20
):
    """Tìm kiếm full-text"""
    if articles_col is None:
        raise HTTPException(503, "Database không khả dụng")
    
    docs = articles_col.find(
        {"$text": {"$search": q}, "is_hcmc": True},
        {"score": {"$meta": "textScore"}}
    ).sort([("score", {"$meta": "textScore"})]).limit(limit)
    
    results = []
    for doc in docs:
        results.append({
            "id": str(doc["_id"]),
            "title": doc["title"],
            "summary": doc.get("summary", "")[:200],
            "source": doc["source_name"],
            "category": doc.get("category", "general"),
            "published_date": doc["published_date"].isoformat(),
            "relevance_score": doc.get("score", 0),
            "url": doc["url"]
        })
    
    return {
        "query": q,
        "results": results,
        "count": len(results)
    }

@app.get("/trending")
def get_trending_topics(days: int = 7, top_n: int = 10):
    """Các chủ đề nổi bật trong N ngày qua"""
    if articles_col is None:
        raise HTTPException(503, "Database không khả dụng")
    
    # Lấy bài viết gần đây
    recent_docs = articles_col.find({
        "is_hcmc": True,
        "published_date": {"$gte": datetime.now() - timedelta(days=days)}
    })
    
    # Đếm từ khóa
    all_text = ""
    for doc in recent_docs:
        all_text += doc["title"] + " " + doc["content"]
    
    # Trích xuất từ khóa
    words = re.findall(r'\w+', all_text.lower())
    
    # Stopwords
    stopwords = {
        'và', 'của', 'có', 'được', 'là', 'trong', 'với', 'cho', 'các', 
        'một', 'này', 'đã', 'từ', 'những', 'để', 'người', 'không', 'như',
        'về', 'theo', 'năm', 'tại', 'đến', 'khi', 'ngày', 'trên', 'sau'
    }
    
    words = [w for w in words if w not in stopwords and len(w) > 3]
    
    # Top keywords
    keyword_freq = Counter(words).most_common(top_n)
    
    trending = [{"keyword": word, "count": count} for word, count in keyword_freq]
    
    return {
        "period_days": days,
        "trending_keywords": trending
    }

@app.post("/regenerate-summary/{article_id}")
def regenerate_summary(article_id: str, num_sentences: int = 3):
    """Tạo lại tóm tắt với thuật toán mới"""
    if articles_col is None:
        raise HTTPException(503, "Database không khả dụng")
    
    doc = articles_col.find_one({"_id": ObjectId(article_id)})
    if doc is None:
        raise HTTPException(404, "Không tìm thấy bài viết")
    
    # Tạo summary mới
    new_summary = AdvancedSummarizer.summarize(
        doc["content"], 
        num_sentences=num_sentences,
        method="textrank"
    )
    
    # Cập nhật DB
    articles_col.update_one(
        {"_id": doc["_id"]},
        {"$set": {"summary": new_summary, "summary_updated_at": datetime.now()}}
    )
    
    return {
        "id": str(doc["_id"]),
        "old_summary": doc.get("summary", ""),
        "new_summary": new_summary
    }

@app.get("/export")
def export_articles(
    format: str = Query("json", description="json hoặc csv"),
    category: Optional[str] = None,
    from_date: Optional[str] = None,
    to_date: Optional[str] = None
):
    """Export dữ liệu"""
    if articles_col is None:
        raise HTTPException(503, "Database không khả dụng")
    
    query = {"is_hcmc": True}
    
    if category:
        query["category"] = category
    
    date_query = {}
    if from_date:
        date_query["$gte"] = datetime.strptime(from_date, "%Y-%m-%d")
    if to_date:
        date_query["$lte"] = datetime.strptime(to_date, "%Y-%m-%d")
    if date_query:
        query["published_date"] = date_query
    
    docs = articles_col.find(query).sort("published_date", -1)
    
    articles = []
    for doc in docs:
        articles.append({
            "id": str(doc["_id"]),
            "title": doc["title"],
            "source": doc["source_name"],
            "category": doc.get("category", "general"),
            "summary": doc.get("summary", ""),
            "published_date": doc["published_date"].isoformat(),
            "url": doc["url"]
        })
    
    if format == "csv":
        # Simple CSV format
        csv_data = "ID,Title,Source,Category,Date,URL\n"
        for art in articles:
            csv_data += f"{art['id']},\"{art['title']}\",{art['source']},{art['category']},{art['published_date']},{art['url']}\n"
        
        return {"format": "csv", "data": csv_data, "count": len(articles)}
    
    return {"format": "json", "articles": articles, "count": len(articles)}

# ==================== CHẠY NHANH (SIMPLE MODE) ====================

def quick_scrape(num_articles_per_source: int = 5):
    """Chạy scrape đơn giản - không cần sys.argv"""
    print(f"\n{'='*70}")
    print(f"🚀 TP.HCM TRAFFIC NEWS INTELLIGENCE SYSTEM 2025")
    print(f"{'='*70}")
    print(f"📅 Thời gian: {START_DATE.date()} → {CURRENT_DATE.date()}")
    print(f"📰 Đang scrape {num_articles_per_source} bài/nguồn...")
    print(f"{'='*70}\n")
    
    results = scrape_all_sources_2025(max_per_source=num_articles_per_source)
    
    print(f"\n{'='*70}")
    print(f"✅ HOÀN THÀNH!")
    print(f"{'='*70}")
    print(f"📊 Kết quả:")
    print(f"   • Tổng URLs: {results['grand_total']['urls']}")
    print(f"   • Đã scrape: {results['grand_total']['scraped']}")
    print(f"   • Đã lưu DB: {results['grand_total']['saved']}")
    print(f"   • Trùng lặp: {results['grand_total']['duplicates']}")
    
    for source_key, stats in results["sources"].items():
        name = NEWS_SOURCES[source_key]["name"]
        print(f"\n   🔹 {name}: {stats['saved']} bài")
    
    print(f"\n{'='*70}")
    print(f"💡 Xem dữ liệu:")
    print(f"   python {__file__} stats")
    print(f"   hoặc chạy API: uvicorn {__file__.replace('.py', '')}:app --reload")
    print(f"{'='*70}\n")
    
    return results

def show_stats_simple():
    """Hiển thị thống kê đơn giản"""
    if articles_col is None:
        print("❌ Không kết nối được MongoDB")
        return
    
    total = articles_col.count_documents({"is_hcmc": True})
    
    print(f"\n{'='*70}")
    print(f"📊 THỐNG KÊ TỔNG QUAN")
    print(f"{'='*70}")
    print(f"Tổng số bài viết về TP.HCM: {total}")
    
    # By category
    print(f"\n📂 Theo danh mục:")
    for category in ["projects", "infrastructure", "issues", "planning", "transport", "general"]:
        count = articles_col.count_documents({"is_hcmc": True, "category": category})
        if count > 0:
            cat_name = {
                "projects": "Dự án",
                "infrastructure": "Hạ tầng", 
                "issues": "Vấn đề",
                "planning": "Quy hoạch",
                "transport": "Vận tải",
                "general": "Chung"
            }.get(category, category)
            print(f"   • {cat_name}: {count} bài")
    
    # By source
    print(f"\n📰 Theo nguồn:")
    for source_key, source in NEWS_SOURCES.items():
        count = articles_col.count_documents({"is_hcmc": True, "source": source_key})
        if count > 0:
            print(f"   • {source['name']}: {count} bài")
    
    # Recent
    recent = articles_col.count_documents({
        "is_hcmc": True,
        "published_date": {"$gte": datetime.now() - timedelta(days=7)}
    })
    print(f"\n📅 7 ngày gần đây: {recent} bài")
    
    print(f"\n{'='*70}\n")

# ==================== CLI MODE ====================

if __name__ == "__main__":
    import sys
    
    # Kiểm tra có argument không
    if len(sys.argv) > 1:
        command = sys.argv[1]
        
        if command == "scrape":
            # Scrape all sources
            max_articles = int(sys.argv[2]) if len(sys.argv) > 2 else 50
            results = scrape_all_sources_2025(max_per_source=max_articles)
            
            print(f"\n{'='*70}")
            print(f"📊 KẾT QUẢ TỔNG HỢP")
            print(f"{'='*70}")
            print(f"Tổng URLs: {results['grand_total']['urls']}")
            print(f"Đã scrape: {results['grand_total']['scraped']}")
            print(f"Đã lưu: {results['grand_total']['saved']}")
            print(f"Trùng lặp: {results['grand_total']['duplicates']}")
            
            print(f"\n📈 Chi tiết theo nguồn:")
            for source_key, stats in results["sources"].items():
                name = NEWS_SOURCES[source_key]["name"]
                print(f"\n  🔹 {name}")
                print(f"     URLs: {stats['total_urls']}")
                print(f"     Đã lưu: {stats['saved']}/{stats['scraped']}")
                print(f"     Không phải HCM: {stats.get('not_hcmc', 0)}")
                print(f"     Bài cũ: {stats.get('old_articles', 0)}")
            
            print(f"\n{'='*70}")
        
        elif command == "stats":
            # Show statistics
            show_stats_simple()
        
        else:
            print(f"❌ Lệnh không hợp lệ: {command}")
            print(f"Sử dụng: python {sys.argv[0]} [scrape|stats]")
    
    else:
        # Không có argument - chạy mode mặc định
        print(f"\n{'='*70}")
        print(f"🚀 TP.HCM TRAFFIC NEWS INTELLIGENCE SYSTEM 2025")
        print(f"{'='*70}")
        print(f"📅 Thời gian: {START_DATE.date()} → {CURRENT_DATE.date()}")
        print(f"📰 Nguồn tin: {len(NEWS_SOURCES)} báo")
        print(f"🤖 Thuật toán: TextRank + TF-IDF")
        print(f"{'='*70}\n")
        
        print("Chọn chế độ:")
        print("  [1] Scrape nhanh (5 bài/nguồn)")
        print("  [2] Xem thống kê")
        print("  [3] Hướng dẫn sử dụng")
        
        try:
            choice = input("\nNhập lựa chọn (1/2/3): ").strip()
            
            if choice == "1":
                quick_scrape(num_articles_per_source=5)
            elif choice == "2":
                show_stats_simple()
            else:
                print(f"\n📖 HƯỚNG DẪN SỬ DỤNG:")
                print(f"\n  1️⃣  Scrape dữ liệu:")
                print(f"     python {sys.argv[0]} scrape [số_bài]")
                print(f"     Ví dụ: python {sys.argv[0]} scrape 50")
                
                print(f"\n  2️⃣  Xem thống kê:")
                print(f"     python {sys.argv[0]} stats")
                
                print(f"\n  3️⃣  Chạy API server:")
                print(f"     uvicorn {sys.argv[0].replace('.py', '')}:app --reload")
                
                print(f"\n  4️⃣  Test API:")
                print(f"     curl http://localhost:8000/stats")
                print(f"     curl http://localhost:8000/articles?category=projects")
                print(f"     curl http://localhost:8000/search?q=metro")
                
                print(f"\n{'='*70}\n")
        
        except KeyboardInterrupt:
            print(f"\n\n👋 Tạm biệt!\n")

# ==================== Thêm vào cuối file ====================
def interactive_mode():
    """Chạy interactive để tránh lỗi ipykernel"""
    print("\n" + "="*70)
    print("🚀 TP.HCM TRAFFIC NEWS SCRAPER")
    print("="*70)
    
    print("\n[1] Scrape 5 bài/nguồn")
    print("[2] Xem thống kê")
    
    choice = input("\nChọn (1/2): ").strip()
    
    if choice == "1":
        quick_scrape(5)
    elif choice == "2":
        show_stats_simple()

# Chạy interactive mode thay vì dùng sys.argv
interactive_mode()

✓ Đã kết nối MongoDB với full-text search
❌ Lệnh không hợp lệ: --f=c:\Users\ASUS\AppData\Roaming\jupyter\runtime\kernel-v31c77da465df6157aa337d65d287250ab347656fe.json
Sử dụng: python C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py [scrape|stats]

🚀 TP.HCM TRAFFIC NEWS SCRAPER

[1] Scrape 5 bài/nguồn
[2] Xem thống kê

🚀 TP.HCM TRAFFIC NEWS INTELLIGENCE SYSTEM 2025
📅 Thời gian: 2025-01-01 → 2025-10-09
📰 Đang scrape 5 bài/nguồn...


🚀 CHIẾN DỊCH SCRAPING: GIAO THÔNG TP.HCM 2025
   Thời gian: 2025-01-01 → 2025-10-09


🔍 BẮT ĐẦU: Báo Lao Động

📡 Bước 1: Lấy URLs từ RSS...
  📡 Đang đọc RSS: https://laodong.vn/rss/giao-thong.rss
  ✓ Tìm thấy 0 URLs từ RSS
  📡 Đang đọc RSS: https://laodong.vn/rss/xa-hoi.rss
  ✓ Tìm thấy 0 URLs từ RSS

📄 Bước 2: Lấy URLs từ trang danh sách...
  📄 Trang 1: https://laodong.vn/giao-thong/...
  ✓ Tìm thấy 7 URLs
  📄 Trang 2: https://laodong.vn/giao-thong/trang-2.htm...
  ✓ Tìm thấy 0 URLs

📊 Tổng cộng: 4 URLs duy nhất

📰 Bước 3: 

NameError: name '__file__' is not defined

GIAI ĐOẠN 1: THU THẬP DỮ LIỆU THÔ (RAW DATA COLLECTION)
=======================================================
Mục tiêu: Scrape bài báo từ nhiều nguồn và lưu vào MongoDB collection "raw_data"
Không làm sạch, không lọc - chỉ thu thập thô

In [ ]:
"""
GIAI ĐOẠN 1: THU THẬP DỮ LIỆU THÔ (RAW DATA COLLECTION)
=======================================================
Mục tiêu: Scrape bài báo từ nhiều nguồn và lưu vào MongoDB collection "raw_data"
Không làm sạch, không lọc - chỉ thu thập thô
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient
from datetime import datetime
import feedparser
import time
import hashlib

# ==================== CẤU HÌNH ====================

# Cấu hình MongoDB
MONGODB_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "hcmc_traffic_summarization"
RAW_COLLECTION = "raw_data"  # Collection cho dữ liệu thô

# Cấu hình nguồn tin
NEWS_SOURCES = {
    "laodong": {
        "name": "Báo Lao Động",
        "base_url": "https://laodong.vn",
        "rss_feeds": ["https://laodong.vn/rss/giao-thong.rss"],
        "selectors": {
            "title": "h1.detail__title, h1",
            "content": "div.detail-content-body, div.detail__content",
            "date": "span.time-update, time"
        }
    },
    "vnexpress": {
        "name": "VnExpress",
        "base_url": "https://vnexpress.net",
        "rss_feeds": ["https://vnexpress.net/rss/giao-thong.rss"],
        "selectors": {
            "title": "h1.title-detail",
            "content": "article.fck_detail, div.fck_detail",
            "date": "span.date"
        }
    },
    "tuoitre": {
        "name": "Báo Tuổi Trẻ",
        "base_url": "https://tuoitre.vn",
        "rss_feeds": ["https://tuoitre.vn/rss/giao-thong.rss"],
        "selectors": {
            "title": "h1.detail-title",
            "content": "div.detail-content, div#main-detail-content",
            "date": "div.date-time"
        }
    }
}

# ==================== KẾT NỐI DATABASE ====================

def connect_mongodb():
    """
    Kết nối MongoDB và tạo collection raw_data
    
    Returns:
        Collection object hoặc None nếu lỗi
    """
    try:
        client = MongoClient(MONGODB_URI, serverSelectionTimeoutMS=5000)
        client.server_info()  # Test connection
        
        db = client[DATABASE_NAME]
        raw_col = db[RAW_COLLECTION]
        
        # Tạo index để tránh trùng lặp URL
        raw_col.create_index([("url", 1)], unique=True)
        raw_col.create_index([("scraped_at", -1)])
        raw_col.create_index([("source", 1)])
        
        print(f"✅ Đã kết nối MongoDB: {DATABASE_NAME}.{RAW_COLLECTION}")
        return raw_col
    
    except Exception as e:
        print(f"❌ Lỗi kết nối MongoDB: {e}")
        return None

# ==================== SCRAPING FUNCTIONS ====================

def init_selenium_driver():
    """
    Khởi tạo Selenium WebDriver (Edge headless)
    
    Returns:
        WebDriver object
    """
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
    
    driver = webdriver.Edge(options=options)
    driver.set_page_load_timeout(30)
    return driver

def get_urls_from_rss(rss_url, max_items=50):
    """
    Lấy danh sách URLs từ RSS feed
    
    Args:
        rss_url: URL của RSS feed
        max_items: Số lượng bài tối đa
    
    Returns:
        List of URLs
    """
    try:
        print(f"  📡 Đang đọc RSS: {rss_url}")
        feed = feedparser.parse(rss_url)
        
        urls = []
        for entry in feed.entries[:max_items]:
            url = entry.get("link", "")
            if url:
                urls.append(url)
        
        print(f"  ✅ Tìm thấy {len(urls)} URLs từ RSS")
        return urls
    
    except Exception as e:
        print(f"  ❌ Lỗi đọc RSS: {e}")
        return []

def scrape_article_raw(url, source_key, driver):
    """
    Scrape 1 bài báo và trả về dữ liệu THÔ (không làm sạch)
    
    Args:
        url: URL bài báo
        source_key: Key của nguồn trong NEWS_SOURCES
        driver: Selenium WebDriver
    
    Returns:
        Dict chứa dữ liệu thô hoặc None nếu lỗi
    """
    source = NEWS_SOURCES[source_key]
    
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        
        # Lấy title (raw - không làm sạch)
        try:
            title_elem = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, source["selectors"]["title"]))
            )
            title = title_elem.text
        except:
            title = driver.title
        
        # Lấy content (raw - bao gồm cả HTML)
        try:
            content_elem = driver.find_element(By.CSS_SELECTOR, source["selectors"]["content"])
            # Lấy cả innerHTML để giữ cấu trúc
            content_html = content_elem.get_attribute("innerHTML")
            content_text = content_elem.text
        except:
            content_html = driver.find_element(By.TAG_NAME, "body").get_attribute("innerHTML")
            content_text = driver.find_element(By.TAG_NAME, "body").text
        
        # Lấy ngày đăng (raw string)
        date_raw = ""
        if "date" in source["selectors"]:
            try:
                date_elem = driver.find_element(By.CSS_SELECTOR, source["selectors"]["date"])
                date_raw = date_elem.text
            except:
                pass
        
        # Tạo hash để phát hiện duplicate sau này
        content_hash = hashlib.md5(content_text.encode('utf-8')).hexdigest()
        
        return {
            "url": url,
            "source": source_key,
            "source_name": source["name"],
            "title": title,  # RAW title
            "content_text": content_text,  # RAW text
            "content_html": content_html,  # RAW HTML
            "date_raw": date_raw,  # RAW date string
            "content_hash": content_hash,
            "scraped_at": datetime.now(),
            "processing_status": "raw"  # Đánh dấu chưa xử lý
        }
    
    except Exception as e:
        print(f"  ❌ Lỗi scrape {url}: {e}")
        return None

# ==================== MAIN SCRAPING PIPELINE ====================

def scrape_source(source_key, raw_collection, max_articles=50):
    """
    Scrape 1 nguồn tin và lưu vào MongoDB
    
    Args:
        source_key: Key của nguồn trong NEWS_SOURCES
        raw_collection: MongoDB collection object
        max_articles: Số lượng bài tối đa
    
    Returns:
        Dict chứa thống kê
    """
    source = NEWS_SOURCES[source_key]
    print(f"\n{'='*60}")
    print(f"🔍 Đang scrape: {source['name']}")
    print(f"{'='*60}")
    
    # Bước 1: Lấy URLs từ RSS
    all_urls = []
    for rss_url in source.get("rss_feeds", []):
        urls = get_urls_from_rss(rss_url, max_items=max_articles)
        all_urls.extend(urls)
    
    # Loại bỏ duplicate URLs
    all_urls = list(set(all_urls))
    print(f"\n📊 Tổng cộng: {len(all_urls)} URLs duy nhất")
    
    # Bước 2: Scrape từng bài
    driver = None
    stats = {
        "total_urls": len(all_urls),
        "scraped_success": 0,
        "saved_to_db": 0,
        "duplicates": 0,
        "errors": 0
    }
    
    try:
        driver = init_selenium_driver()
        
        for idx, url in enumerate(all_urls[:max_articles], 1):
            print(f"\n[{idx}/{min(len(all_urls), max_articles)}] {url[:60]}...")
            
            # Kiểm tra đã tồn tại trong DB chưa
            if raw_collection.find_one({"url": url}):
                print(f"  ⏭️  Đã tồn tại trong DB")
                stats["duplicates"] += 1
                continue
            
            # Scrape bài báo
            article_data = scrape_article_raw(url, source_key, driver)
            
            if article_data is None:
                stats["errors"] += 1
                continue
            
            stats["scraped_success"] += 1
            
            # Lưu vào MongoDB (raw data)
            try:
                raw_collection.insert_one(article_data)
                print(f"  ✅ Đã lưu raw data: {article_data['title'][:50]}...")
                stats["saved_to_db"] += 1
            except Exception as e:
                print(f"  ❌ Lỗi lưu DB: {e}")
                stats["duplicates"] += 1
            
            time.sleep(2)  # Tránh spam server
    
    finally:
        if driver:
            driver.quit()
    
    return stats

def scrape_all_sources(max_per_source=50):
    """
    Scrape tất cả nguồn tin
    
    Args:
        max_per_source: Số bài tối đa mỗi nguồn
    
    Returns:
        Dict chứa thống kê tổng hợp
    """
    print(f"\n{'='*70}")
    print(f"🚀 GIAI ĐOẠN 1: THU THẬP DỮ LIỆU THÔ")
    print(f"{'='*70}\n")
    
    # Kết nối MongoDB
    raw_collection = connect_mongodb()
    if raw_collection is None:
        print("❌ Không thể kết nối MongoDB. Dừng scraping.")
        return None
    
    # Scrape từng nguồn
    total_stats = {
        "sources": {},
        "grand_total": {
            "scraped": 0,
            "saved": 0,
            "duplicates": 0,
            "errors": 0
        }
    }
    
    for source_key in NEWS_SOURCES.keys():
        stats = scrape_source(source_key, raw_collection, max_per_source)
        total_stats["sources"][source_key] = stats
        
        total_stats["grand_total"]["scraped"] += stats["scraped_success"]
        total_stats["grand_total"]["saved"] += stats["saved_to_db"]
        total_stats["grand_total"]["duplicates"] += stats["duplicates"]
        total_stats["grand_total"]["errors"] += stats["errors"]
    
    # In báo cáo tổng kết
    print(f"\n{'='*70}")
    print(f"✅ HOÀN THÀNH GIAI ĐOẠN 1")
    print(f"{'='*70}")
    print(f"📊 Thống kê tổng hợp:")
    print(f"   • Đã scrape thành công: {total_stats['grand_total']['scraped']}")
    print(f"   • Đã lưu vào raw_data: {total_stats['grand_total']['saved']}")
    print(f"   • Trùng lặp: {total_stats['grand_total']['duplicates']}")
    print(f"   • Lỗi: {total_stats['grand_total']['errors']}")
    
    print(f"\n📈 Chi tiết theo nguồn:")
    for source_key, stats in total_stats["sources"].items():
        name = NEWS_SOURCES[source_key]["name"]
        print(f"   • {name}: {stats['saved_to_db']} bài")
    
    print(f"\n{'='*70}")
    print(f"➡️  Tiếp theo: Chạy 2_data_filtering.py để lọc & gán nhãn")
    print(f"{'='*70}\n")
    
    return total_stats

# ==================== CHẠY CHƯƠNG TRÌNH ====================

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) > 1:
        # Chạy với tham số: python 1_data_collection.py 100
        max_articles = int(sys.argv[1])
    else:
        # Mặc định: 50 bài/nguồn
        max_articles = 50
    
    print(f"⚙️  Cấu hình: Tối đa {max_articles} bài/nguồn\n")
    
    # Bắt đầu scraping
    results = scrape_all_sources(max_per_source=max_articles)
    
    if results:
        print(f"\n✅ Dữ liệu thô đã được lưu vào MongoDB collection '{RAW_COLLECTION}'")
        print(f"📊 Tổng số bài: {results['grand_total']['saved']}")

ValueError: invalid literal for int() with base 10: '--f=c:\\Users\\ASUS\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3e88060ec6ce0abe0108c4929c27dbcc5f1d608e1.json'

GIAI ĐOẠN 2: LỌC VÀ GÁN NHÃN (DATA FILTERING & LABELING)
=========================================================
Mục tiêu: 
- Lọc chỉ giữ bài viết về giao thông TP.HCM năm 2025
- Gán nhãn category tự động (projects/infrastructure/issues/planning/transport)
- Phát hiện và loại bỏ duplicate

In [ ]:
"""
GIAI ĐOẠN 2: LỌC VÀ GÁN NHÃN (DATA FILTERING & LABELING)
=========================================================
Mục tiêu: 
- Lọc chỉ giữ bài viết về giao thông TP.HCM năm 2025
- Gán nhãn category tự động (projects/infrastructure/issues/planning/transport)
- Phát hiện và loại bỏ duplicate
"""

from pymongo import MongoClient
from datetime import datetime
import re
from collections import Counter

# ==================== CẤU HÌNH ====================

# MongoDB
MONGODB_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "hcmc_traffic_summarization"
RAW_COLLECTION = "raw_data"
FILTERED_COLLECTION = "filtered_data"  # Collection sau khi lọc

# Thời gian lọc
START_DATE = datetime(2025, 1, 1)
END_DATE = datetime(2025, 12, 31)

# Từ khóa TP.HCM (để xác định bài viết thuộc TP.HCM)
HCMC_KEYWORDS = [
    "tp.hcm", "tp hcm", "tphcm", "hcmc",
    "hồ chí minh", "ho chi minh",
    "sài gòn", "saigon",
    "thành phố hồ chí minh",
    # Địa danh cụ thể
    "quận 1", "quận 2", "quận 3", "quận 4", "quận 5",
    "bình thạnh", "tân bình", "phú nhuận", "gò vấp",
    "thủ đức", "bình tân", "tân phú",
    "cầu sài gòn", "cầu phú mỹ", "cầu bình triệu",
    "đại lộ võ văn kiệt", "đại lộ đông tây"
]

# Từ khóa giao thông (để xác định bài viết về giao thông)
TRAFFIC_KEYWORDS = [
    # Cơ sở hạ tầng
    "giao thông", "metro", "tàu điện", "cao tốc", "cầu", "đường",
    "vành đai", "brt", "đường sắt", "đường bộ",
    
    # Dự án
    "dự án", "khởi công", "hoàn thành", "đầu tư", "xây dựng",
    "triển khai", "thi công",
    
    # Vấn đề
    "kẹt xe", "ùn tắc", "tai nạn", "giao thông", "ngập nước",
    "ổ gà", "hư hỏng",
    
    # Phương tiện
    "xe buýt", "xe bus", "taxi", "grab", "xe máy", "ô tô",
    "vận tải", "xe điện",
    
    # Quy hoạch
    "quy hoạch", "kế hoạch", "phê duyệt", "điều chỉnh"
]

# Danh mục và từ khóa đặc trưng
CATEGORY_KEYWORDS = {
    "projects": {
        "name": "Dự án giao thông",
        "keywords": [
            "metro", "tàu điện", "cao tốc", "cầu", "khởi công",
            "hoàn thành", "dự án", "đầu tư", "xây dựng", "triển khai",
            "thi công", "đường vành đai", "brt"
        ]
    },
    "infrastructure": {
        "name": "Hạ tầng giao thông",
        "keywords": [
            "hạ tầng", "đường bộ", "đường sắt", "đường thủy",
            "sân bay", "cảng", "bến xe", "trạm", "nút giao thông",
            "cơ sở hạ tầng", "hệ thống"
        ]
    },
    "issues": {
        "name": "Vấn đề giao thông",
        "keywords": [
            "kẹt xe", "ùn tắc", "tai nạn", "ngập nước", "ổ gà",
            "hư hỏng", "ách tắc", "tắc đường", "va chạm",
            "đụng độ", "sự cố"
        ]
    },
    "planning": {
        "name": "Quy hoạch giao thông",
        "keywords": [
            "quy hoạch", "kế hoạch", "phê duyệt", "điều chỉnh",
            "đề án", "chính sách", "pháp luật", "quyết định",
            "chỉ đạo", "nghiên cứu"
        ]
    },
    "transport": {
        "name": "Phương tiện vận tải",
        "keywords": [
            "xe buýt", "xe bus", "taxi", "grab", "xe máy", "ô tô",
            "vận tải", "xe điện", "xe công cộng", "xe khách",
            "tuyến xe", "điểm dừng"
        ]
    }
}

# ==================== KẾT NỐI DATABASE ====================

def connect_mongodb():
    """
    Kết nối MongoDB và tạo collection filtered_data
    
    Returns:
        Tuple (raw_collection, filtered_collection)
    """
    try:
        client = MongoClient(MONGODB_URI, serverSelectionTimeoutMS=5000)
        client.server_info()
        
        db = client[DATABASE_NAME]
        raw_col = db[RAW_COLLECTION]
        filtered_col = db[FILTERED_COLLECTION]
        
        # Tạo index
        filtered_col.create_index([("url", 1)], unique=True)
        filtered_col.create_index([("published_date", -1)])
        filtered_col.create_index([("category", 1)])
        filtered_col.create_index([("is_hcmc", 1)])
        
        print(f"✅ Đã kết nối MongoDB")
        print(f"   Raw data: {raw_col.count_documents({})}")
        return raw_col, filtered_col
    
    except Exception as e:
        print(f"❌ Lỗi kết nối MongoDB: {e}")
        return None, None

# ==================== FILTERING FUNCTIONS ====================

def is_hcmc_related(text):
    """
    Kiểm tra bài viết có liên quan đến TP.HCM không
    
    Args:
        text: Nội dung cần kiểm tra (title + content)
    
    Returns:
        bool: True nếu liên quan TP.HCM
    """
    text_lower = text.lower()
    
    # Đếm số lần xuất hiện từ khóa TP.HCM
    hcmc_count = sum(1 for kw in HCMC_KEYWORDS if kw in text_lower)
    
    # Quy tắc: Phải có ít nhất 2 từ khóa TP.HCM HOẶC 1 từ khóa xuất hiện nhiều lần
    if hcmc_count >= 2:
        return True
    
    # Kiểm tra từ khóa chính xuất hiện nhiều lần
    for kw in ["tp.hcm", "tphcm", "hồ chí minh", "sài gòn"]:
        if text_lower.count(kw) >= 2:
            return True
    
    return False

def is_traffic_related(text):
    """
    Kiểm tra bài viết có liên quan đến giao thông không
    
    Args:
        text: Nội dung cần kiểm tra
    
    Returns:
        bool: True nếu liên quan giao thông
    """
    text_lower = text.lower()
    
    # Đếm số lần xuất hiện từ khóa giao thông
    traffic_count = sum(1 for kw in TRAFFIC_KEYWORDS if kw in text_lower)
    
    # Quy tắc: Phải có ít nhất 3 từ khóa giao thông
    return traffic_count >= 3

def parse_date_from_raw(date_raw):
    """
    Parse ngày tháng từ chuỗi raw
    
    Args:
        date_raw: Chuỗi ngày tháng chưa xử lý
    
    Returns:
        datetime object hoặc None
    """
    if not date_raw:
        return None
    
    # Danh sách các format phổ biến
    formats = [
        "%d/%m/%Y %H:%M",
        "%d/%m/%Y, %H:%M",
        "%d-%m-%Y %H:%M",
        "%d/%m/%Y",
        "%d-%m-%Y",
        "%Y-%m-%d",
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(date_raw.strip(), fmt)
        except:
            continue
    
    # Thử trích xuất ngày từ chuỗi bằng regex
    match = re.search(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})', date_raw)
    if match:
        day, month, year = match.groups()
        try:
            return datetime(int(year), int(month), int(day))
        except:
            pass
    
    return None

def categorize_article(title, content):
    """
    Phân loại bài viết tự động theo category
    
    Args:
        title: Tiêu đề bài viết
        content: Nội dung bài viết
    
    Returns:
        str: Category key (projects/infrastructure/issues/planning/transport/general)
    """
    text = (title + " " + content).lower()
    
    # Đếm số lần xuất hiện từ khóa của mỗi category
    scores = {}
    for cat_key, cat_data in CATEGORY_KEYWORDS.items():
        score = sum(1 for kw in cat_data["keywords"] if kw in text)
        scores[cat_key] = score
    
    # Lấy category có điểm cao nhất
    if max(scores.values()) == 0:
        return "general"
    
    return max(scores, key=scores.get)

def detect_duplicate(content_hash, filtered_col):
    """
    Phát hiện bài viết trùng lặp dựa vào content_hash
    
    Args:
        content_hash: MD5 hash của content
        filtered_col: MongoDB collection
    
    Returns:
        bool: True nếu là duplicate
    """
    existing = filtered_col.find_one({"content_hash": content_hash})
    return existing is not None

# ==================== MAIN FILTERING PIPELINE ====================

def filter_article(raw_doc, filtered_col):
    """
    Lọc và xử lý 1 bài viết raw
    
    Args:
        raw_doc: Document từ raw_data collection
        filtered_col: Collection để lưu kết quả
    
    Returns:
        Dict với status và message
    """
    url = raw_doc.get("url", "")
    title = raw_doc.get("title", "")
    content = raw_doc.get("content_text", "")
    
    # Bước 1: Kiểm tra duplicate
    if detect_duplicate(raw_doc.get("content_hash"), filtered_col):
        return {"status": "duplicate", "reason": "Nội dung trùng lặp"}
    
    # Bước 2: Kiểm tra liên quan TP.HCM
    if not is_hcmc_related(title + " " + content):
        return {"status": "not_hcmc", "reason": "Không liên quan TP.HCM"}
    
    # Bước 3: Kiểm tra liên quan giao thông
    if not is_traffic_related(title + " " + content):
        return {"status": "not_traffic", "reason": "Không liên quan giao thông"}
    
    # Bước 4: Parse ngày đăng
    published_date = parse_date_from_raw(raw_doc.get("date_raw", ""))
    
    # Nếu không parse được ngày, dùng scraped_at
    if published_date is None:
        published_date = raw_doc.get("scraped_at", datetime.now())
    
    # Bước 5: Kiểm tra năm 2025
    if published_date.year != 2025:
        return {"status": "wrong_year", "reason": f"Năm {published_date.year}, không phải 2025"}
    
    # Bước 6: Gán category
    category = categorize_article(title, content)
    
    # Tạo document mới cho filtered_data
    filtered_doc = {
        "url": url,
        "source": raw_doc.get("source"),
        "source_name": raw_doc.get("source_name"),
        "title": title,
        "content_text": content,
        "content_html": raw_doc.get("content_html"),
        "content_hash": raw_doc.get("content_hash"),
        "published_date": published_date,
        "category": category,
        "category_name": CATEGORY_KEYWORDS.get(category, {}).get("name", "Chung"),
        "is_hcmc": True,
        "is_traffic": True,
        "scraped_at": raw_doc.get("scraped_at"),
        "filtered_at": datetime.now(),
        "processing_status": "filtered"  # Đánh dấu đã lọc
    }
    
    return {"status": "pass", "doc": filtered_doc, "category": category}

def run_filtering(batch_size=100):
    """
    Chạy quá trình lọc cho toàn bộ raw_data
    
    Args:
        batch_size: Số lượng document xử lý mỗi lần
    
    Returns:
        Dict chứa thống kê
    """
    print(f"\n{'='*70}")
    print(f"🔍 GIAI ĐOẠN 2: LỌC VÀ GÁN NHÃN")
    print(f"{'='*70}\n")
    
    # Kết nối MongoDB
    raw_col, filtered_col = connect_mongodb()
    if raw_col is None or filtered_col is None:
        print("❌ Không thể kết nối MongoDB")
        return None
    
    # Lấy tất cả raw documents chưa xử lý
    total_raw = raw_col.count_documents({"processing_status": "raw"})
    print(f"📊 Tổng số bài raw chưa xử lý: {total_raw}")
    
    stats = {
        "total_processed": 0,
        "passed": 0,
        "not_hcmc": 0,
        "not_traffic": 0,
        "wrong_year": 0,
        "duplicates": 0,
        "errors": 0,
        "by_category": Counter()
    }
    
    # Xử lý từng batch
    cursor = raw_col.find({"processing_status": "raw"}).batch_size(batch_size)
    
    for idx, raw_doc in enumerate(cursor, 1):
        print(f"\r[{idx}/{total_raw}] Đang xử lý...", end="", flush=True)
        
        stats["total_processed"] += 1
        
        try:
            # Lọc bài viết
            result = filter_article(raw_doc, filtered_col)
            
            if result["status"] == "pass":
                # Lưu vào filtered_data
                try:
                    filtered_col.insert_one(result["doc"])
                    stats["passed"] += 1
                    stats["by_category"][result["category"]] += 1
                    
                    # Cập nhật trạng thái trong raw_data
                    raw_col.update_one(
                        {"_id": raw_doc["_id"]},
                        {"$set": {"processing_status": "filtered"}}
                    )
                except Exception as e:
                    print(f"\n  ❌ Lỗi lưu DB: {e}")
                    stats["errors"] += 1
            
            elif result["status"] == "duplicate":
                stats["duplicates"] += 1
            elif result["status"] == "not_hcmc":
                stats["not_hcmc"] += 1
            elif result["status"] == "not_traffic":
                stats["not_traffic"] += 1
            elif result["status"] == "wrong_year":
                stats["wrong_year"] += 1
        
        except Exception as e:
            print(f"\n  ❌ Lỗi xử lý: {e}")
            stats["errors"] += 1
    
    print()  # Xuống dòng sau progress bar
    
    # In báo cáo
    print(f"\n{'='*70}")
    print(f"✅ HOÀN THÀNH GIAI ĐOẠN 2")
    print(f"{'='*70}")
    print(f"📊 Thống kê:")
    print(f"   • Tổng số đã xử lý: {stats['total_processed']}")
    print(f"   • Đã lọc thành công: {stats['passed']} ({stats['passed']/stats['total_processed']*100:.1f}%)")
    print(f"   • Không phải TP.HCM: {stats['not_hcmc']}")
    print(f"   • Không phải giao thông: {stats['not_traffic']}")
    print(f"   • Không phải năm 2025: {stats['wrong_year']}")
    print(f"   • Trùng lặp: {stats['duplicates']}")
    print(f"   • Lỗi: {stats['errors']}")
    
    print(f"\n📂 Phân loại theo category:")
    for cat_key, count in stats["by_category"].most_common():
        cat_name = CATEGORY_KEYWORDS.get(cat_key, {}).get("name", cat_key)
        print(f"   • {cat_name}: {count} bài")
    
    print(f"\n{'='*70}")
    print(f"➡️  Tiếp theo: Chạy 3_data_cleaning.py để làm sạch dữ liệu")
    print(f"{'='*70}\n")
    
    return stats

# ==================== CHẠY CHƯƠNG TRÌNH ====================

if __name__ == "__main__":
    # Chạy filtering
    results = run_filtering(batch_size=100)
    
    if results:
        print(f"\n✅ Dữ liệu đã lọc được lưu vào collection '{FILTERED_COLLECTION}'")
        print(f"📊 Số bài đạt chuẩn: {results['passed']}")

GIAI ĐOẠN 3: LÀM SẠCH DỮ LIỆU (DATA CLEANING)
==============================================
Mục tiêu:
- Loại bỏ HTML tags, ký tự đặc biệt, quảng cáo
- Chuẩn hóa unicode, dấu câu, khoảng trắng
- Tách câu (sentence segmentation)
- Lưu vào collection "clean_data"

In [ ]:
"""
GIAI ĐOẠN 3: LÀM SẠCH DỮ LIỆU (DATA CLEANING)
==============================================
Mục tiêu:
- Loại bỏ HTML tags, ký tự đặc biệt, quảng cáo
- Chuẩn hóa unicode, dấu câu, khoảng trắng
- Tách câu (sentence segmentation)
- Lưu vào collection "clean_data"
"""

from pymongo import MongoClient
from datetime import datetime
import re
from bs4 import BeautifulSoup
import unicodedata

# ==================== CẤU HÌNH ====================

# MongoDB
MONGODB_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "hcmc_traffic_summarization"
FILTERED_COLLECTION = "filtered_data"
CLEAN_COLLECTION = "clean_data"  # Collection sau khi làm sạch

# Các mẫu quảng cáo/spam cần loại bỏ
SPAM_PATTERNS = [
    r"quảng cáo",
    r"liên hệ:?\s*\d",
    r"hotline:?\s*\d",
    r"đăng ký ngay",
    r"click here",
    r"xem thêm tại",
    r"theo\s+\w+\.vn",  # Nguồn trích dẫn cuối bài
]

# Các từ/cụm từ không mang thông tin (stopwords mở rộng)
NOISE_PHRASES = [
    "theo đó", "bên cạnh đó", "ngoài ra",
    "như đã biết", "được biết", "có thể nói",
    "cũng theo", "đồng thời", "trong khi đó"
]

# ==================== KẾT NỐI DATABASE ====================

def connect_mongodb():
    """
    Kết nối MongoDB và tạo collection clean_data
    
    Returns:
        Tuple (filtered_collection, clean_collection)
    """
    try:
        client = MongoClient(MONGODB_URI, serverSelectionTimeoutMS=5000)
        client.server_info()
        
        db = client[DATABASE_NAME]
        filtered_col = db[FILTERED_COLLECTION]
        clean_col = db[CLEAN_COLLECTION]
        
        # Tạo index
        clean_col.create_index([("url", 1)], unique=True)
        clean_col.create_index([("published_date", -1)])
        clean_col.create_index([("category", 1)])
        
        print(f"✅ Đã kết nối MongoDB")
        print(f"   Filtered data: {filtered_col.count_documents({})}")
        return filtered_col, clean_col
    
    except Exception as e:
        print(f"❌ Lỗi kết nối MongoDB: {e}")
        return None, None

# ==================== TEXT CLEANING FUNCTIONS ====================

def remove_html(html_text):
    """
    Loại bỏ HTML tags
    
    Args:
        html_text: Chuỗi chứa HTML
    
    Returns:
        str: Text thuần túy
    """
    if not html_text:
        return ""
    
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Loại bỏ script và style tags
    for script in soup(["script", "style"]):
        script.decompose()
    
    # Lấy text
    text = soup.get_text()
    
    return text

def normalize_unicode(text):
    """
    Chuẩn hóa Unicode (NFC normalization)
    Ví dụ: "café" có thể được encode theo nhiều cách
    
    Args:
        text: Chuỗi cần chuẩn hóa
    
    Returns:
        str: Chuỗi đã chuẩn hóa
    """
    return unicodedata.normalize('NFC', text)

def remove_special_chars(text):
    """
    Loại bỏ ký tự đặc biệt không cần thiết
    
    Args:
        text: Chuỗi cần xử lý
    
    Returns:
        str: Chuỗi đã làm sạch
    """
    # Giữ lại: chữ cái, số, dấu câu cơ bản, khoảng trắng
    text = re.sub(r'[^\w\s.,!?;:()\-–—\'\"°%/]', ' ', text)
    
    # Loại bỏ emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    return text

def remove_spam_content(text):
    """
    Loại bỏ nội dung quảng cáo/spam
    
    Args:
        text: Chuỗi cần xử lý
    
    Returns:
        str: Chuỗi đã loại bỏ spam
    """
    for pattern in SPAM_PATTERNS:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    return text

def normalize_whitespace(text):
    """
    Chuẩn hóa khoảng trắng
    - Loại bỏ khoảng trắng thừa
    - Loại bỏ dòng trống
    
    Args:
        text: Chuỗi cần xử lý
    
    Returns:
        str: Chuỗi đã chuẩn hóa
    """
    # Thay thế nhiều khoảng trắng bằng 1
    text = re.sub(r'\s+', ' ', text)
    
    # Loại bỏ khoảng trắng đầu/cuối mỗi dòng
    lines = [line.strip() for line in text.split('\n')]
    
    # Loại bỏ dòng trống
    lines = [line for line in lines if line]
    
    return '\n'.join(lines)

def normalize_punctuation(text):
    """
    Chuẩn hóa dấu câu
    
    Args:
        text: Chuỗi cần xử lý
    
    Returns:
        str: Chuỗi đã chuẩn hóa dấu câu
    """
    # Chuẩn hóa dấu ngoặc kép
    text = text.replace('"', '"').replace('"', '"')
    text = text.replace("'", "'").replace("'", "'")
    
    # Chuẩn hóa dấu gạch ngang
    text = text.replace('–', '-').replace('—', '-')
    
    # Thêm khoảng trắng sau dấu câu nếu thiếu
    text = re.sub(r'([.,!?;:])([^\s])', r'\1 \2', text)
    
    # Loại bỏ khoảng trắng trước dấu câu
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    
    return text

def segment_sentences(text):
    """
    Tách câu trong văn bản tiếng Việt
    
    Args:
        text: Chuỗi văn bản
    
    Returns:
        List[str]: Danh sách các câu
    """
    # Tách theo dấu câu kết thúc
    sentences = re.split(r'[.!?]\s+', text)
    
    # Làm sạch và lọc câu
    cleaned_sentences = []
    for sent in sentences:
        sent = sent.strip()
        
        # Chỉ giữ câu có độ dài hợp lý (20-500 ký tự)
        if 20 <= len(sent) <= 500:
            # Loại bỏ câu chỉ chứa số hoặc ký tự đặc biệt
            if re.search(r'[a-zA-Zàáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđ]', sent):
                cleaned_sentences.append(sent)
    
    return cleaned_sentences

def clean_title(title):
    """
    Làm sạch tiêu đề
    
    Args:
        title: Tiêu đề cần làm sạch
    
    Returns:
        str: Tiêu đề đã làm sạch
    """
    # Loại bỏ emoji và ký tự đặc biệt
    title = remove_special_chars(title)
    
    # Chuẩn hóa
    title = normalize_unicode(title)
    title = normalize_whitespace(title)
    title = normalize_punctuation(title)
    
    # Loại bỏ các từ như [Video], [Infographic]
    title = re.sub(r'\[[\w\s]+\]', '', title)
    
    return title.strip()

def clean_content(content_text, content_html):
    """
    Làm sạch nội dung bài viết
    
    Args:
        content_text: Nội dung dạng text thuần
        content_html: Nội dung dạng HTML
    
    Returns:
        Dict chứa nội dung đã làm sạch và sentences
    """
    # Ưu tiên xử lý từ HTML để có kết quả tốt hơn
    if content_html:
        text = remove_html(content_html)
    else:
        text = content_text
    
    # Pipeline làm sạch
    text = normalize_unicode(text)
    text = remove_spam_content(text)
    text = remove_special_chars(text)
    text = normalize_whitespace(text)
    text = normalize_punctuation(text)
    
    # Tách câu
    sentences = segment_sentences(text)
    
    # Gộp lại thành văn bản liền mạch
    clean_text = '. '.join(sentences) + '.'
    
    return {
        "content_clean": clean_text,
        "sentences": sentences,
        "num_sentences": len(sentences),
        "num_words": len(clean_text.split())
    }

# ==================== MAIN CLEANING PIPELINE ====================

def clean_article(filtered_doc):
    """
    Làm sạch 1 bài viết
    
    Args:
        filtered_doc: Document từ filtered_data collection
    
    Returns:
        Dict: Document đã làm sạch
    """
    # Làm sạch title
    clean_title_text = clean_title(filtered_doc.get("title", ""))
    
    # Làm sạch content
    content_result = clean_content(
        filtered_doc.get("content_text", ""),
        filtered_doc.get("content_html", "")
    )
    
    # Tạo document mới
    clean_doc = {
        "url": filtered_doc.get("url"),
        "source": filtered_doc.get("source"),
        "source_name": filtered_doc.get("source_name"),
        
        # Dữ liệu đã làm sạch
        "title_clean": clean_title_text,
        "content_clean": content_result["content_clean"],
        "sentences": content_result["sentences"],
        
        # Metadata
        "num_sentences": content_result["num_sentences"],
        "num_words": content_result["num_words"],
        "content_hash": filtered_doc.get("content_hash"),
        
        # Phân loại
        "published_date": filtered_doc.get("published_date"),
        "category": filtered_doc.get("category"),
        "category_name": filtered_doc.get("category_name"),
        "is_hcmc": filtered_doc.get("is_hcmc"),
        "is_traffic": filtered_doc.get("is_traffic"),
        
        # Timestamps
        "scraped_at": filtered_doc.get("scraped_at"),
        "filtered_at": filtered_doc.get("filtered_at"),
        "cleaned_at": datetime.now(),
        "processing_status": "cleaned"  # Đánh dấu đã làm sạch
    }
    
    return clean_doc

def run_cleaning(batch_size=100):
    """
    Chạy quá trình làm sạch cho toàn bộ filtered_data
    
    Args:
        batch_size: Số lượng document xử lý mỗi lần
    
    Returns:
        Dict chứa thống kê
    """
    print(f"\n{'='*70}")
    print(f"🧹 GIAI ĐOẠN 3: LÀM SẠCH DỮ LIỆU")
    print(f"{'='*70}\n")
    
    # Kết nối MongoDB
    filtered_col, clean_col = connect_mongodb()
    if filtered_col is None or clean_col is None:
        print("❌ Không thể kết nối MongoDB")
        return None
    
    # Lấy tất cả filtered documents chưa làm sạch
    total_filtered = filtered_col.count_documents({"processing_status": "filtered"})
    print(f"📊 Tổng số bài cần làm sạch: {total_filtered}")
    
    stats = {
        "total_processed": 0,
        "cleaned": 0,
        "errors": 0,
        "total_sentences": 0,
        "total_words": 0
    }
    
    # Xử lý từng batch
    cursor = filtered_col.find({"processing_status": "filtered"}).batch_size(batch_size)
    
    for idx, filtered_doc in enumerate(cursor, 1):
        print(f"\r[{idx}/{total_filtered}] Đang xử lý...", end="", flush=True)
        
        stats["total_processed"] += 1
        
        try:
            # Làm sạch bài viết
            clean_doc = clean_article(filtered_doc)
            
            # Lưu vào clean_data
            try:
                clean_col.insert_one(clean_doc)
                stats["cleaned"] += 1
                stats["total_sentences"] += clean_doc["num_sentences"]
                stats["total_words"] += clean_doc["num_words"]
                
                # Cập nhật trạng thái trong filtered_data
                filtered_col.update_one(
                    {"_id": filtered_doc["_id"]},
                    {"$set": {"processing_status": "cleaned"}}
                )
            except Exception as e:
                print(f"\n  ❌ Lỗi lưu DB: {e}")
                stats["errors"] += 1
        
        except Exception as e:
            print(f"\n  ❌ Lỗi xử lý: {e}")
            stats["errors"] += 1
    
    print()  # Xuống dòng sau progress bar
    
    # In báo cáo
    print(f"\n{'='*70}")
    print(f"✅ HOÀN THÀNH GIAI ĐOẠN 3")
    print(f"{'='*70}")
    print(f"📊 Thống kê:")
    print(f"   • Tổng số đã xử lý: {stats['total_processed']}")
    print(f"   • Đã làm sạch thành công: {stats['cleaned']}")
    print(f"   • Lỗi: {stats['errors']}")
    print(f"   • Tổng số câu: {stats['total_sentences']}")
    print(f"   • Tổng số từ: {stats['total_words']}")
    
    if stats['cleaned'] > 0:
        avg_sentences = stats['total_sentences'] / stats['cleaned']
        avg_words = stats['total_words'] / stats['cleaned']
        print(f"   • Trung bình: {avg_sentences:.1f} câu/bài, {avg_words:.0f} từ/bài")
    
    print(f"\n{'='*70}")
    print(f"➡️  Tiếp theo: Chạy 4_model_training.py để huấn luyện mô hình")
    print(f"{'='*70}\n")
    
    return stats

# ==================== CHẠY CHƯƠNG TRÌNH ====================

if __name__ == "__main__":
    # Chạy cleaning
    results = run_cleaning(batch_size=100)
    
    if results:
        print(f"\n✅ Dữ liệu đã làm sạch được lưu vào collection '{CLEAN_COLLECTION}'")
        print(f"📊 Số bài đã xử lý: {results['cleaned']}")

GIAI ĐOẠN 4: HUẤN LUYỆN MÔ HÌNH (MODEL TRAINING)
=================================================
Mục tiêu:
- Sử dụng ViT5 (Vietnamese T5) cho tóm tắt abstractive
- Fine-tune trên dữ liệu giao thông TP.HCM
- Chia train/val/test: 70/15/15
- Đánh giá bằng ROUGE, BLEU

In [ ]:
"""
GIAI ĐOẠN 4: HUẤN LUYỆN MÔ HÌNH (MODEL TRAINING)
=================================================
Mục tiêu:
- Sử dụng ViT5 (Vietnamese T5) cho tóm tắt abstractive
- Fine-tune trên dữ liệu giao thông TP.HCM
- Chia train/val/test: 70/15/15
- Đánh giá bằng ROUGE, BLEU
"""

from pymongo import MongoClient
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
import torch
import numpy as np
from datetime import datetime
import json
import os
from sklearn.model_selection import train_test_split

# Đánh giá metrics
try:
    from rouge_score import rouge_scorer
    ROUGE_AVAILABLE = True
except:
    ROUGE_AVAILABLE = False
    print("⚠️  Chưa cài rouge_score. Chạy: pip install rouge-score")

# ==================== CẤU HÌNH ====================

# MongoDB
MONGODB_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "hcmc_traffic_summarization"
CLEAN_COLLECTION = "clean_data"

# Mô hình
MODEL_NAME = "VietAI/vit5-base"  # Mô hình ViT5 pre-trained
OUTPUT_DIR = "./models/vit5_hcmc_traffic"  # Thư mục lưu model

# Training hyperparameters
MAX_INPUT_LENGTH = 512  # Độ dài tối đa của input (bài báo)
MAX_TARGET_LENGTH = 128  # Độ dài tối đa của summary
BATCH_SIZE = 4  # Giảm nếu thiếu RAM/VRAM
LEARNING_RATE = 5e-5
NUM_EPOCHS = 5
WARMUP_STEPS = 500

# Data split
TRAIN_RATIO = 0.70
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# ==================== KẾT NỐI DATABASE ====================

def connect_mongodb():
    """
    Kết nối MongoDB và lấy clean_data
    
    Returns:
        Collection object hoặc None
    """
    try:
        client = MongoClient(MONGODB_URI, serverSelectionTimeoutMS=5000)
        client.server_info()
        
        db = client[DATABASE_NAME]
        clean_col = db[CLEAN_COLLECTION]
        
        total = clean_col.count_documents({})
        print(f"✅ Đã kết nối MongoDB")
        print(f"   Clean data: {total} bài")
        return clean_col
    
    except Exception as e:
        print(f"❌ Lỗi kết nối MongoDB: {e}")
        return None

# ==================== CHUẨN BỊ DỮ LIỆU ====================

def load_data_from_mongodb(clean_col):
    """
    Load dữ liệu từ MongoDB và chuẩn bị cho training
    
    Args:
        clean_col: MongoDB collection
    
    Returns:
        List of dicts với format {content, summary}
    """
    print(f"\n📥 Đang load dữ liệu từ MongoDB...")
    
    data = []
    cursor = clean_col.find({
        "processing_status": "cleaned",
        "num_sentences": {"$gte": 5}  # Chỉ lấy bài có ít nhất 5 câu
    })
    
    for doc in cursor:
        # Input: Nội dung bài báo đã làm sạch
        content = doc.get("content_clean", "")
        
        # Target: Tạo summary từ 3 câu đầu tiên (pseudo-label)
        # LƯU Ý: Đây là phương pháp tạm thời. Tốt nhất là có human-labeled summaries
        sentences = doc.get("sentences", [])
        if len(sentences) >= 5:
            # Lấy 3 câu đầu làm summary (extractive)
            summary = ". ".join(sentences[:3]) + "."
            
            data.append({
                "content": content,
                "summary": summary,
                "category": doc.get("category", "general"),
                "url": doc.get("url", "")
            })
    
    print(f"✅ Đã load {len(data)} bài có đủ điều kiện")
    return data

def split_dataset(data):
    """
    Chia dataset thành train/val/test
    
    Args:
        data: List of dicts
    
    Returns:
        Tuple (train_data, val_data, test_data)
    """
    print(f"\n🔀 Đang chia dataset...")
    
    # Chia train + temp (val + test)
    train_data, temp_data = train_test_split(
        data, 
        test_size=(VAL_RATIO + TEST_RATIO),
        random_state=42,
        shuffle=True
    )
    
    # Chia val và test
    val_ratio_adjusted = VAL_RATIO / (VAL_RATIO + TEST_RATIO)
    val_data, test_data = train_test_split(
        temp_data,
        test_size=(1 - val_ratio_adjusted),
        random_state=42
    )
    
    print(f"✅ Đã chia dataset:")
    print(f"   • Train: {len(train_data)} bài ({len(train_data)/len(data)*100:.1f}%)")
    print(f"   • Val: {len(val_data)} bài ({len(val_data)/len(data)*100:.1f}%)")
    print(f"   • Test: {len(test_data)} bài ({len(test_data)/len(data)*100:.1f}%)")
    
    return train_data, val_data, test_data

def prepare_dataset(data, tokenizer):
    """
    Chuẩn bị dataset cho Hugging Face Trainer
    
    Args:
        data: List of dicts
        tokenizer: Tokenizer object
    
    Returns:
        Dataset object
    """
    # Chuyển sang format của Hugging Face
    dataset_dict = {
        "content": [item["content"] for item in data],
        "summary": [item["summary"] for item in data]
    }
    
    dataset = Dataset.from_dict(dataset_dict)
    
    # Tokenize
    def tokenize_function(examples):
        # Tokenize input (content)
        model_inputs = tokenizer(
            examples["content"],
            max_length=MAX_INPUT_LENGTH,
            truncation=True,
            padding="max_length"
        )
        
        # Tokenize target (summary)
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                examples["summary"],
                max_length=MAX_TARGET_LENGTH,
                truncation=True,
                padding="max_length"
            )
        
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset.column_names
    )
    
    return tokenized_dataset

# ==================== METRICS ====================

def compute_metrics(eval_pred):
    """
    Tính toán metrics cho evaluation
    
    Args:
        eval_pred: Tuple (predictions, labels)
    
    Returns:
        Dict chứa các metrics
    """
    predictions, labels = eval_pred
    
    # Decode predictions và labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 trong labels (padding tokens)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Tính ROUGE scores
    if ROUGE_AVAILABLE:
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
        
        rouge1_scores = []
        rouge2_scores = []
        rougeL_scores = []
        
        for pred, label in zip(decoded_preds, decoded_labels):
            scores = scorer.score(label, pred)
            rouge1_scores.append(scores['rouge1'].fmeasure)
            rouge2_scores.append(scores['rouge2'].fmeasure)
            rougeL_scores.append(scores['rougeL'].fmeasure)
        
        return {
            "rouge1": np.mean(rouge1_scores),
            "rouge2": np.mean(rouge2_scores),
            "rougeL": np.mean(rougeL_scores)
        }
    else:
        return {}

# ==================== MAIN TRAINING PIPELINE ====================

def train_model():
    """
    Huấn luyện mô hình ViT5 cho tóm tắt
    
    Returns:
        Trainer object
    """
    print(f"\n{'='*70}")
    print(f"🤖 GIAI ĐOẠN 4: HUẤN LUYỆN MÔ HÌNH")
    print(f"{'='*70}\n")
    
    # Bước 1: Load dữ liệu từ MongoDB
    clean_col = connect_mongodb()
    if clean_col is None:
        print("❌ Không thể kết nối MongoDB")
        return None
    
    data = load_data_from_mongodb(clean_col)
    
    if len(data) < 100:
        print(f"⚠️  Cảnh báo: Chỉ có {len(data)} bài. Cần ít nhất 100 bài để train tốt.")
        response = input("Tiếp tục? (y/n): ")
        if response.lower() != 'y':
            return None
    
    # Bước 2: Chia dataset
    train_data, val_data, test_data = split_dataset(data)
    
    # Lưu test set để evaluation sau
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    with open(f"{OUTPUT_DIR}/test_data.json", "w", encoding="utf-8") as f:
        json.dump(test_data, f, ensure_ascii=False, indent=2)
    print(f"✅ Đã lưu test set: {OUTPUT_DIR}/test_data.json")
    
    # Bước 3: Load tokenizer và model
    print(f"\n📦 Đang load mô hình {MODEL_NAME}...")
    
    global tokenizer  # Để dùng trong compute_metrics
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    
    print(f"✅ Đã load mô hình")
    
    # Kiểm tra GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"💻 Device: {device}")
    
    if device == "cpu":
        print("⚠️  Cảnh báo: Đang dùng CPU. Training sẽ rất chậm. Khuyến nghị dùng GPU.")
    
    # Bước 4: Chuẩn bị datasets
    print(f"\n🔨 Đang tokenize datasets...")
    train_dataset = prepare_dataset(train_data, tokenizer)
    val_dataset = prepare_dataset(val_data, tokenizer)
    print(f"✅ Đã tokenize xong")
    
    # Bước 5: Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=OUTPUT_DIR,
        evaluation_strategy="epoch",  # Evaluate mỗi epoch
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=NUM_EPOCHS,
        warmup_steps=WARMUP_STEPS,
        weight_decay=0.01,
        save_total_limit=2,  # Chỉ giữ 2 checkpoint tốt nhất
        predict_with_generate=True,  # Generate text khi evaluate
        fp16=True if device == "cuda" else False,  # Mixed precision training
        logging_dir=f"{OUTPUT_DIR}/logs",
        logging_steps=50,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="rougeL" if ROUGE_AVAILABLE else "loss",
        greater_is_better=True if ROUGE_AVAILABLE else False,
    )
    
    # Data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    # Bước 6: Khởi tạo Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics if ROUGE_AVAILABLE else None
    )
    
    # Bước 7: Bắt đầu training
    print(f"\n{'='*70}")
    print(f"🚀 BẮT ĐẦU TRAINING")
    print(f"{'='*70}\n")
    print(f"Cấu hình:")
    print(f"   • Epochs: {NUM_EPOCHS}")
    print(f"   • Batch size: {BATCH_SIZE}")
    print(f"   • Learning rate: {LEARNING_RATE}")
    print(f"   • Device: {device}")
    print(f"\n⏱️  Thời gian dự kiến: {len(train_data) * NUM_EPOCHS / BATCH_SIZE / 60:.1f} phút")
    print(f"\nĐang training...\n")
    
    train_result = trainer.train()
    
    # Bước 8: Lưu model
    print(f"\n💾 Đang lưu model...")
    trainer.save_model(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    
    # Lưu training metrics
    metrics = train_result.metrics
    with open(f"{OUTPUT_DIR}/train_metrics.json", "w") as f:
        json.dump(metrics, f, indent=2)
    
    print(f"\n{'='*70}")
    print(f"✅ HOÀN THÀNH TRAINING")
    print(f"{'='*70}")
    print(f"📊 Training metrics:")
    for key, value in metrics.items():
        print(f"   • {key}: {value:.4f}")
    
    print(f"\n💾 Model đã được lưu tại: {OUTPUT_DIR}")
    print(f"\n{'='*70}")
    print(f"➡️  Tiếp theo: Chạy 5_model_evaluation.py để đánh giá chi tiết")
    print(f"{'='*70}\n")
    
    return trainer

# ==================== CHẠY CHƯƠNG TRÌNH ====================

if __name__ == "__main__":
    import sys
    
    print(f"\n⚠️  LƯU Ý QUAN TRỌNG:")
    print(f"   • Training mô hình cần GPU mạnh (ít nhất 8GB VRAM)")
    print(f"   • Thời gian: 1-3 giờ tùy số lượng bài và GPU")
    print(f"   • Cần cài đặt: pip install transformers datasets torch rouge-score")
    print(f"   • Hiện tại đang dùng pseudo-labels (3 câu đầu). Tốt nhất có human-labeled data\n")
    
    # Hỏi xác nhận
    if len(sys.argv) > 1 and sys.argv[1] == "--auto":
        # Auto mode: không hỏi
        pass
    else:
        response = input("Bạn có muốn tiếp tục training? (y/n): ")
        if response.lower() != 'y':
            print("❌ Đã hủy training")
            sys.exit(0)
    
    # Bắt đầu training
    trainer = train_model()
    
    if trainer:
        print(f"\n✅ Training thành công!")
        print(f"📁 Model được lưu tại: {OUTPUT_DIR}")

GIAI ĐOẠN 5: ĐÁNH GIÁ MÔ HÌNH (MODEL EVALUATION)
=================================================
Mục tiêu:
- Đánh giá mô hình trên test set
- Tính ROUGE, BLEU scores
- Kiểm tra overfitting
- Phân tích lỗi

In [ ]:
"""
GIAI ĐOẠN 5: ĐÁNH GIÁ MÔ HÌNH (MODEL EVALUATION)
=================================================
Mục tiêu:
- Đánh giá mô hình trên test set
- Tính ROUGE, BLEU scores
- Kiểm tra overfitting
- Phân tích lỗi
"""

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import json
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# Metrics
try:
    from rouge_score import rouge_scorer
    ROUGE_AVAILABLE = True
except:
    ROUGE_AVAILABLE = False
    print("⚠️  Cần cài: pip install rouge-score")

try:
    from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
    BLEU_AVAILABLE = True
except:
    BLEU_AVAILABLE = False
    print("⚠️  Cần cài: pip install nltk")

# ==================== CẤU HÌNH ====================

MODEL_DIR = "./models/vit5_hcmc_traffic"  # Thư mục chứa model đã train
TEST_DATA_PATH = f"{MODEL_DIR}/test_data.json"  # Test set

# Generation parameters
MAX_LENGTH = 128
MIN_LENGTH = 30
NUM_BEAMS = 4  # Beam search
NO_REPEAT_NGRAM_SIZE = 3  # Tránh lặp n-gram

# ==================== LOAD MODEL ====================

def load_model():
    """
    Load mô hình đã fine-tune
    
    Returns:
        Tuple (model, tokenizer, device)
    """
    print(f"\n📦 Đang load mô hình từ {MODEL_DIR}...")
    
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
        model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR)
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = model.to(device)
        model.eval()  # Evaluation mode
        
        print(f"✅ Đã load mô hình")
        print(f"💻 Device: {device}")
        
        return model, tokenizer, device
    
    except Exception as e:
        print(f"❌ Lỗi load model: {e}")
        return None, None, None

# ==================== GENERATE SUMMARY ====================

def generate_summary(content, model, tokenizer, device):
    """
    Tạo summary từ content
    
    Args:
        content: Nội dung bài báo
        model: Model object
        tokenizer: Tokenizer object
        device: Device (cuda/cpu)
    
    Returns:
        str: Summary đã generate
    """
    # Tokenize input
    inputs = tokenizer(
        content,
        max_length=512,
        truncation=True,
        return_tensors="pt"
    ).to(device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=MAX_LENGTH,
            min_length=MIN_LENGTH,
            num_beams=NUM_BEAMS,
            no_repeat_ngram_size=NO_REPEAT_NGRAM_SIZE,
            early_stopping=True
        )
    
    # Decode
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return summary

# ==================== METRICS CALCULATION ====================

def calculate_rouge(predictions, references):
    """
    Tính ROUGE scores
    
    Args:
        predictions: List of generated summaries
        references: List of reference summaries
    
    Returns:
        Dict chứa ROUGE scores
    """
    if not ROUGE_AVAILABLE:
        return {}
    
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)
    
    return {
        "rouge1": {
            "mean": np.mean(rouge1_scores),
            "std": np.std(rouge1_scores),
            "min": np.min(rouge1_scores),
            "max": np.max(rouge1_scores)
        },
        "rouge2": {
            "mean": np.mean(rouge2_scores),
            "std": np.std(rouge2_scores),
            "min": np.min(rouge2_scores),
            "max": np.max(rouge2_scores)
        },
        "rougeL": {
            "mean": np.mean(rougeL_scores),
            "std": np.std(rougeL_scores),
            "min": np.min(rougeL_scores),
            "max": np.max(rougeL_scores)
        }
    }

def calculate_bleu(predictions, references):
    """
    Tính BLEU scores
    
    Args:
        predictions: List of generated summaries
        references: List of reference summaries
    
    Returns:
        Dict chứa BLEU scores
    """
    if not BLEU_AVAILABLE:
        return {}
    
    smoothing = SmoothingFunction().method1
    bleu_scores = []
    
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.split()
        ref_tokens = [ref.split()]  # BLEU expects list of references
        
        try:
            score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothing)
            bleu_scores.append(score)
        except:
            bleu_scores.append(0.0)
    
    return {
        "bleu": {
            "mean": np.mean(bleu_scores),
            "std": np.std(bleu_scores),
            "min": np.min(bleu_scores),
            "max": np.max(bleu_scores)
        }
    }

def calculate_length_stats(predictions, references):
    """
    Thống kê độ dài summaries
    
    Args:
        predictions: List of generated summaries
        references: List of reference summaries
    
    Returns:
        Dict chứa thống kê độ dài
    """
    pred_lengths = [len(p.split()) for p in predictions]
    ref_lengths = [len(r.split()) for r in references]
    
    return {
        "prediction_length": {
            "mean": np.mean(pred_lengths),
            "std": np.std(pred_lengths),
            "min": np.min(pred_lengths),
            "max": np.max(pred_lengths)
        },
        "reference_length": {
            "mean": np.mean(ref_lengths),
            "std": np.std(ref_lengths)
        }
    }

# ==================== OVERFITTING CHECK ====================

def check_overfitting():
    """
    Kiểm tra overfitting bằng cách so sánh train và val loss
    
    Returns:
        Dict chứa phân tích overfitting
    """
    try:
        with open(f"{MODEL_DIR}/train_metrics.json", "r") as f:
            train_metrics = json.load(f)
        
        # Lấy train và eval loss từ trainer_state.json
        with open(f"{MODEL_DIR}/trainer_state.json", "r") as f:
            trainer_state = json.load(f)
        
        log_history = trainer_state.get("log_history", [])
        
        train_losses = []
        eval_losses = []
        
        for log in log_history:
            if "loss" in log:
                train_losses.append(log["loss"])
            if "eval_loss" in log:
                eval_losses.append(log["eval_loss"])
        
        # Phân tích overfitting
        if len(train_losses) > 0 and len(eval_losses) > 0:
            final_train_loss = train_losses[-1]
            final_eval_loss = eval_losses[-1]
            
            gap = final_eval_loss - final_train_loss
            gap_percent = (gap / final_train_loss) * 100
            
            if gap_percent > 20:
                status = "HIGH_OVERFITTING"
                message = "Mô hình bị overfit nghiêm trọng. Cần giảm model complexity hoặc tăng data."
            elif gap_percent > 10:
                status = "MODERATE_OVERFITTING"
                message = "Mô hình có dấu hiệu overfit nhẹ. Có thể chấp nhận được."
            else:
                status = "GOOD"
                message = "Mô hình học tốt, không bị overfit."
            
            return {
                "status": status,
                "message": message,
                "final_train_loss": final_train_loss,
                "final_eval_loss": final_eval_loss,
                "gap": gap,
                "gap_percent": gap_percent,
                "train_losses": train_losses,
                "eval_losses": eval_losses
            }
        
        return {"status": "UNKNOWN", "message": "Không đủ dữ liệu để phân tích"}
    
    except Exception as e:
        return {"status": "ERROR", "message": str(e)}

# ==================== MAIN EVALUATION ====================

def run_evaluation():
    """
    Chạy evaluation trên test set
    
    Returns:
        Dict chứa kết quả đánh giá
    """
    print(f"\n{'='*70}")
    print(f"📊 GIAI ĐOẠN 5: ĐÁNH GIÁ MÔ HÌNH")
    print(f"{'='*70}\n")
    
    # Bước 1: Load model
    model, tokenizer, device = load_model()
    if model is None:
        print("❌ Không thể load model. Hãy chạy 4_model_training.py trước.")
        return None
    
    # Bước 2: Load test data
    print(f"\n📥 Đang load test data từ {TEST_DATA_PATH}...")
    try:
        with open(TEST_DATA_PATH, "r", encoding="utf-8") as f:
            test_data = json.load(f)
        print(f"✅ Đã load {len(test_data)} bài test")
    except Exception as e:
        print(f"❌ Lỗi load test data: {e}")
        return None
    
    # Bước 3: Generate summaries
    print(f"\n🤖 Đang generate summaries...")
    predictions = []
    references = []
    
    for idx, item in enumerate(test_data, 1):
        print(f"\r[{idx}/{len(test_data)}] Đang generate...", end="", flush=True)
        
        content = item["content"]
        reference = item["summary"]
        
        # Generate summary
        prediction = generate_summary(content, model, tokenizer, device)
        
        predictions.append(prediction)
        references.append(reference)
    
    print()  # Xuống dòng sau progress bar
    
    # Bước 4: Tính metrics
    print(f"\n📏 Đang tính metrics...")
    
    rouge_scores = calculate_rouge(predictions, references)
    bleu_scores = calculate_bleu(predictions, references)
    length_stats = calculate_length_stats(predictions, references)
    
    # Bước 5: Kiểm tra overfitting
    print(f"\n🔍 Đang kiểm tra overfitting...")
    overfitting_analysis = check_overfitting()
    
    # Bước 6: Tổng hợp kết quả
    results = {
        "test_size": len(test_data),
        "rouge_scores": rouge_scores,
        "bleu_scores": bleu_scores,
        "length_stats": length_stats,
        "overfitting_analysis": overfitting_analysis,
        "evaluated_at": datetime.now().isoformat()
    }
    
    # Lưu kết quả
    results_path = f"{MODEL_DIR}/evaluation_results.json"
    with open(results_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    # Lưu ví dụ predictions
    examples_path = f"{MODEL_DIR}/evaluation_examples.json"
    examples = []
    for i in range(min(10, len(test_data))):  # Lưu 10 ví dụ
        examples.append({
            "content": test_data[i]["content"][:200] + "...",  # Chỉ lưu 200 ký tự đầu
            "reference": references[i],
            "prediction": predictions[i],
            "url": test_data[i].get("url", "")
        })
    
    with open(examples_path, "w", encoding="utf-8") as f:
        json.dump(examples, f, ensure_ascii=False, indent=2)
    
    # In báo cáo
    print(f"\n{'='*70}")
    print(f"✅ HOÀN THÀNH ĐÁNH GIÁ")
    print(f"{'='*70}")
    
    print(f"\n📊 KẾT QUẢ ĐÁNH GIÁ:")
    print(f"\n🎯 ROUGE Scores:")
    if rouge_scores:
        for metric, values in rouge_scores.items():
            print(f"   • {metric.upper()}: {values['mean']:.4f} ± {values['std']:.4f}")
            print(f"     (min: {values['min']:.4f}, max: {values['max']:.4f})")
    else:
        print("   ⚠️  Không tính được ROUGE scores")
    
    print(f"\n📝 BLEU Scores:")
    if bleu_scores:
        print(f"   • BLEU: {bleu_scores['bleu']['mean']:.4f} ± {bleu_scores['bleu']['std']:.4f}")
    else:
        print("   ⚠️  Không tính được BLEU scores")
    
    print(f"\n📏 Độ dài Summary:")
    print(f"   • Generated: {length_stats['prediction_length']['mean']:.1f} từ "
          f"(min: {length_stats['prediction_length']['min']}, "
          f"max: {length_stats['prediction_length']['max']})")
    print(f"   • Reference: {length_stats['reference_length']['mean']:.1f} từ")
    
    print(f"\n🔍 Phân tích Overfitting:")
    print(f"   • Status: {overfitting_analysis.get('status', 'UNKNOWN')}")
    print(f"   • {overfitting_analysis.get('message', 'N/A')}")
    if 'gap_percent' in overfitting_analysis:
        print(f"   • Train/Val gap: {overfitting_analysis['gap_percent']:.2f}%")
    
    # Đánh giá tổng thể
    print(f"\n{'='*70}")
    print(f"💡 ĐÁNH GIÁ TỔNG QUAN:")
    
    if rouge_scores:
        rougeL_mean = rouge_scores['rougeL']['mean']
        
        if rougeL_mean >= 0.4:
            quality = "TỐT"
            emoji = "🟢"
            comment = "Mô hình hoạt động tốt!"
        elif rougeL_mean >= 0.3:
            quality = "KHÁ"
            emoji = "🟡"
            comment = "Mô hình hoạt động ổn. Có thể cải thiện thêm."
        else:
            quality = "CẦN CẢI THIỆN"
            emoji = "🔴"
            comment = "Mô hình cần fine-tune thêm hoặc tăng data."
        
        print(f"{emoji} Chất lượng: {quality}")
        print(f"   {comment}")
    
    print(f"\n📁 Kết quả đã lưu:")
    print(f"   • Chi tiết: {results_path}")
    print(f"   • Ví dụ: {examples_path}")
    
    print(f"\n{'='*70}")
    print(f"➡️  Tiếp theo: Chạy 6_generate_summaries.py để tóm tắt toàn bộ dữ liệu")
    print(f"{'='*70}\n")
    
    return results

# ==================== VẼ BIỂU ĐỒ ====================

def plot_training_curves():
    """
    Vẽ biểu đồ train/val loss theo epoch
    """
    print(f"\n📈 Đang vẽ biểu đồ training curves...")
    
    try:
        overfitting_analysis = check_overfitting()
        
        if overfitting_analysis['status'] not in ['UNKNOWN', 'ERROR']:
            train_losses = overfitting_analysis['train_losses']
            eval_losses = overfitting_analysis['eval_losses']
            
            plt.figure(figsize=(10, 6))
            
            # Train loss
            plt.plot(range(1, len(train_losses) + 1), train_losses, 
                    label='Train Loss', marker='o', linewidth=2)
            
            # Eval loss
            eval_epochs = np.linspace(1, len(train_losses), len(eval_losses))
            plt.plot(eval_epochs, eval_losses, 
                    label='Validation Loss', marker='s', linewidth=2)
            
            plt.xlabel('Epoch', fontsize=12)
            plt.ylabel('Loss', fontsize=12)
            plt.title('Training và Validation Loss theo Epoch', fontsize=14, fontweight='bold')
            plt.legend(fontsize=11)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            
            # Lưu biểu đồ
            plot_path = f"{MODEL_DIR}/training_curves.png"
            plt.savefig(plot_path, dpi=300)
            print(f"✅ Đã lưu biểu đồ: {plot_path}")
            
            plt.close()
        else:
            print(f"⚠️  Không đủ dữ liệu để vẽ biểu đồ")
    
    except Exception as e:
        print(f"❌ Lỗi vẽ biểu đồ: {e}")

# ==================== CHẠY CHƯƠNG TRÌNH ====================

if __name__ == "__main__":
    import sys
    
    # Chạy evaluation
    results = run_evaluation()
    
    if results:
        # Vẽ biểu đồ (optional)
        try:
            plot_training_curves()
        except:
            print("⚠️  Không thể vẽ biểu đồ (cần matplotlib)")
        
        print(f"\n✅ Evaluation hoàn tất!")
        print(f"\n💡 Để xem chi tiết:")
        print(f"   cat {MODEL_DIR}/evaluation_results.json")
        print(f"   cat {MODEL_DIR}/evaluation_examples.json")

GIAI ĐOẠN 6: TẠO TÓM TẮT CHO TOÀN BỘ DỮ LIỆU (GENERATE SUMMARIES)
===================================================================
Mục tiêu:
- Sử dụng mô hình đã fine-tune để tóm tắt tất cả bài trong clean_data
- Thêm phân tích ngữ cảnh giao thông (keywords extraction)
- Lưu vào collection "final_output" với trường "summary" và "analysis"

In [ ]:
"""
GIAI ĐOẠN 6: TẠO TÓM TẮT CHO TOÀN BỘ DỮ LIỆU (GENERATE SUMMARIES)
===================================================================
Mục tiêu:
- Sử dụng mô hình đã fine-tune để tóm tắt tất cả bài trong clean_data
- Thêm phân tích ngữ cảnh giao thông (keywords extraction)
- Lưu vào collection "final_output" với trường "summary" và "analysis"
"""

from pymongo import MongoClient
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
from datetime import datetime
import re
from collections import Counter

# ==================== CẤU HÌNH ====================

# MongoDB
MONGODB_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "hcmc_traffic_summarization"
CLEAN_COLLECTION = "clean_data"
FINAL_COLLECTION = "final_output"  # Collection chứa kết quả cuối cùng

# Model
MODEL_DIR = "./models/vit5_hcmc_traffic"

# Generation parameters
MAX_LENGTH = 128
MIN_LENGTH = 30
NUM_BEAMS = 4
BATCH_SIZE = 8  # Xử lý theo batch để nhanh hơn

# Keywords giao thông để phân tích
TRAFFIC_ENTITIES = {
    "infrastructure": [
        "metro", "tàu điện", "cao tốc", "cầu", "đường vành đai", 
        "brt", "đường sắt", "nút giao thông", "cảng", "sân bay"
    ],
    "locations": [
        "quận 1", "quận 2", "quận 3", "thủ đức", "bình thạnh",
        "tân bình", "gò vấp", "bình tân", "cầu sài gòn"
    ],
    "issues": [
        "kẹt xe", "ùn tắc", "tai nạn", "ngập nước", "ổ gà", "hư hỏng"
    ],
    "projects": [
        "dự án", "khởi công", "hoàn thành", "triển khai", "đầu tư"
    ]
}

# ==================== KẾT NỐI DATABASE ====================

def connect_mongodb():
    """
    Kết nối MongoDB và tạo collection final_output
    
    Returns:
        Tuple (clean_collection, final_collection)
    """
    try:
        client = MongoClient(MONGODB_URI, serverSelectionTimeoutMS=5000)
        client.server_info()
        
        db = client[DATABASE_NAME]
        clean_col = db[CLEAN_COLLECTION]
        final_col = db[FINAL_COLLECTION]
        
        # Tạo index
        final_col.create_index([("url", 1)], unique=True)
        final_col.create_index([("published_date", -1)])
        final_col.create_index([("category", 1)])
        
        # Full-text search index
        final_col.create_index([
            ("title_clean", "text"),
            ("summary", "text"),
            ("analysis", "text")
        ])
        
        print(f"✅ Đã kết nối MongoDB")
        print(f"   Clean data: {clean_col.count_documents({})}")
        return clean_col, final_col
    
    except Exception as e:
        print(f"❌ Lỗi kết nối MongoDB: {e}")
        return None, None

# ==================== LOAD MODEL ====================

def load_model():
    """
    Load mô hình đã fine-tune
    
    Returns:
        Tuple (model, tokenizer, device)
    """
    print(f"\n📦 Đang load mô hình từ {MODEL_DIR}...")
    
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
        model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR)
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = model.to(device)
        model.eval()
        
        print(f"✅ Đã load mô hình")
        print(f"💻 Device: {device}")
        
        return model, tokenizer, device
    
    except Exception as e:
        print(f"❌ Lỗi load model: {e}")
        return None, None, None

# ==================== TEXT ANALYSIS ====================

def extract_keywords(text, top_n=10):
    """
    Trích xuất keywords từ văn bản
    
    Args:
        text: Văn bản cần trích xuất
        top_n: Số lượng keywords
    
    Returns:
        List of keywords
    """
    # Tách từ
    words = re.findall(r'\w+', text.lower())
    
    # Stopwords tiếng Việt
    stopwords = {
        'và', 'của', 'có', 'được', 'là', 'trong', 'với', 'cho', 'các',
        'một', 'này', 'đã', 'từ', 'những', 'để', 'người', 'không', 'như',
        'về', 'theo', 'năm', 'tại', 'đến', 'khi', 'ngày', 'trên', 'sau',
        'vào', 'thì', 'sẽ', 'ra', 'đang', 'nên', 'bị', 'hay', 'nhưng'
    }
    
    # Lọc stopwords và từ ngắn
    words = [w for w in words if w not in stopwords and len(w) > 3]
    
    # Đếm tần suất
    word_freq = Counter(words)
    
    # Lấy top keywords
    keywords = [word for word, _ in word_freq.most_common(top_n)]
    
    return keywords

def analyze_traffic_context(text):
    """
    Phân tích ngữ cảnh giao thông trong bài viết
    
    Args:
        text: Văn bản cần phân tích
    
    Returns:
        Dict chứa phân tích
    """
    text_lower = text.lower()
    
    analysis = {
        "mentioned_infrastructure": [],
        "mentioned_locations": [],
        "mentioned_issues": [],
        "mentioned_projects": [],
        "main_topic": ""
    }
    
    # Phát hiện các entities giao thông
    for category, keywords in TRAFFIC_ENTITIES.items():
        mentioned = [kw for kw in keywords if kw in text_lower]
        
        if category == "infrastructure":
            analysis["mentioned_infrastructure"] = mentioned
        elif category == "locations":
            analysis["mentioned_locations"] = mentioned
        elif category == "issues":
            analysis["mentioned_issues"] = mentioned
        elif category == "projects":
            analysis["mentioned_projects"] = mentioned
    
    # Xác định chủ đề chính
    if analysis["mentioned_projects"]:
        analysis["main_topic"] = "Dự án giao thông"
    elif analysis["mentioned_issues"]:
        analysis["main_topic"] = "Vấn đề giao thông"
    elif analysis["mentioned_infrastructure"]:
        analysis["main_topic"] = "Hạ tầng giao thông"
    else:
        analysis["main_topic"] = "Giao thông chung"
    
    return analysis

def generate_analysis_text(analysis, keywords):
    """
    Tạo văn bản phân tích từ dữ liệu analysis
    
    Args:
        analysis: Dict từ analyze_traffic_context
        keywords: List of keywords
    
    Returns:
        str: Văn bản phân tích
    """
    parts = []
    
    # Chủ đề chính
    parts.append(f"Chủ đề: {analysis['main_topic']}.")
    
    # Hạ tầng được đề cập
    if analysis['mentioned_infrastructure']:
        infra = ", ".join(analysis['mentioned_infrastructure'][:3])
        parts.append(f"Hạ tầng: {infra}.")
    
    # Địa điểm
    if analysis['mentioned_locations']:
        locs = ", ".join(analysis['mentioned_locations'][:3])
        parts.append(f"Khu vực: {locs}.")
    
    # Vấn đề
    if analysis['mentioned_issues']:
        issues = ", ".join(analysis['mentioned_issues'][:3])
        parts.append(f"Vấn đề: {issues}.")
    
    # Keywords
    if keywords:
        kw = ", ".join(keywords[:5])
        parts.append(f"Từ khóa: {kw}.")
    
    return " ".join(parts)

# ==================== GENERATE SUMMARY ====================

def generate_summary(content, model, tokenizer, device):
    """
    Tạo summary từ content
    
    Args:
        content: Nội dung bài báo
        model: Model object
        tokenizer: Tokenizer object
        device: Device (cuda/cpu)
    
    Returns:
        str: Summary đã generate
    """
    # Tokenize
    inputs = tokenizer(
        content,
        max_length=512,
        truncation=True,
        return_tensors="pt"
    ).to(device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=MAX_LENGTH,
            min_length=MIN_LENGTH,
            num_beams=NUM_BEAMS,
            no_repeat_ngram_size=3,
            early_stopping=True
        )
    
    # Decode
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return summary

# ==================== MAIN PIPELINE ====================

def process_article(clean_doc, model, tokenizer, device):
    """
    Xử lý 1 bài viết: tóm tắt + phân tích
    
    Args:
        clean_doc: Document từ clean_data
        model, tokenizer, device: Model components
    
    Returns:
        Dict: Document cho final_output
    """
    content = clean_doc.get("content_clean", "")
    title = clean_doc.get("title_clean", "")
    
    # Bước 1: Generate summary bằng model
    summary = generate_summary(content, model, tokenizer, device)
    
    # Bước 2: Trích xuất keywords
    keywords = extract_keywords(content, top_n=10)
    
    # Bước 3: Phân tích ngữ cảnh giao thông
    analysis_data = analyze_traffic_context(content)
    
    # Bước 4: Tạo văn bản phân tích
    analysis_text = generate_analysis_text(analysis_data, keywords)
    
    # Tạo final document
    final_doc = {
        "url": clean_doc.get("url"),
        "source": clean_doc.get("source"),
        "source_name": clean_doc.get("source_name"),
        
        # Dữ liệu gốc
        "title_clean": title,
        "content_clean": content,  # Giữ lại content gốc
        
        # Kết quả chính: SUMMARY và ANALYSIS
        "summary": summary,  # Tóm tắt bằng AI
        "analysis": analysis_text,  # Phân tích ngữ cảnh
        
        # Metadata bổ sung
        "keywords": keywords,
        "traffic_entities": analysis_data,
        
        # Phân loại
        "published_date": clean_doc.get("published_date"),
        "category": clean_doc.get("category"),
        "category_name": clean_doc.get("category_name"),
        
        # Thống kê
        "num_sentences": clean_doc.get("num_sentences"),
        "num_words": clean_doc.get("num_words"),
        "summary_length": len(summary.split()),
        
        # Timestamps
        "scraped_at": clean_doc.get("scraped_at"),
        "cleaned_at": clean_doc.get("cleaned_at"),
        "summarized_at": datetime.now(),
        "processing_status": "completed"  # Đánh dấu hoàn thành
    }
    
    return final_doc

def run_generation(batch_size=BATCH_SIZE):
    """
    Chạy tóm tắt cho toàn bộ clean_data
    
    Args:
        batch_size: Số lượng bài xử lý mỗi lần
    
    Returns:
        Dict chứa thống kê
    """
    print(f"\n{'='*70}")
    print(f"🚀 GIAI ĐOẠN 6: TẠO TÓM TẮT CHO TOÀN BỘ DỮ LIỆU")
    print(f"{'='*70}\n")
    
    # Bước 1: Kết nối MongoDB
    clean_col, final_col = connect_mongodb()
    if clean_col is None or final_col is None:
        print("❌ Không thể kết nối MongoDB")
        return None
    
    # Bước 2: Load model
    model, tokenizer, device = load_model()
    if model is None:
        print("❌ Không thể load model. Hãy chạy 4_model_training.py trước.")
        return None
    
    # Bước 3: Lấy dữ liệu cần xử lý
    total_clean = clean_col.count_documents({"processing_status": "cleaned"})
    print(f"\n📊 Tổng số bài cần tóm tắt: {total_clean}")
    
    if total_clean == 0:
        print("❌ Không có dữ liệu để xử lý. Hãy chạy 3_data_cleaning.py trước.")
        return None
    
    stats = {
        "total_processed": 0,
        "summarized": 0,
        "errors": 0,
        "total_summary_length": 0
    }
    
    # Bước 4: Xử lý từng bài
    cursor = clean_col.find({"processing_status": "cleaned"})
    
    for idx, clean_doc in enumerate(cursor, 1):
        print(f"\r[{idx}/{total_clean}] Đang xử lý...", end="", flush=True)
        
        stats["total_processed"] += 1
        
        try:
            # Kiểm tra đã tồn tại trong final_output chưa
            url = clean_doc.get("url")
            if final_col.find_one({"url": url}):
                continue  # Bỏ qua nếu đã xử lý
            
            # Process: tóm tắt + phân tích
            final_doc = process_article(clean_doc, model, tokenizer, device)
            
            # Lưu vào final_output
            final_col.insert_one(final_doc)
            
            stats["summarized"] += 1
            stats["total_summary_length"] += final_doc["summary_length"]
            
            # Cập nhật trạng thái trong clean_data
            clean_col.update_one(
                {"_id": clean_doc["_id"]},
                {"$set": {"processing_status": "summarized"}}
            )
        
        except Exception as e:
            print(f"\n  ❌ Lỗi xử lý: {e}")
            stats["errors"] += 1
    
    print()  # Xuống dòng sau progress bar
    
    # In báo cáo
    print(f"\n{'='*70}")
    print(f"✅ HOÀN THÀNH GIAI ĐOẠN 6")
    print(f"{'='*70}")
    print(f"📊 Thống kê:")
    print(f"   • Tổng số đã xử lý: {stats['total_processed']}")
    print(f"   • Đã tóm tắt thành công: {stats['summarized']}")
    print(f"   • Lỗi: {stats['errors']}")
    
    if stats['summarized'] > 0:
        avg_summary_length = stats['total_summary_length'] / stats['summarized']
        print(f"   • Độ dài summary trung bình: {avg_summary_length:.1f} từ")
    
    print(f"\n{'='*70}")
    print(f"🎉 HOÀN THÀNH TẤT CẢ CÁC GIAI ĐOẠN!")
    print(f"{'='*70}")
    print(f"\n📁 Dữ liệu cuối cùng đã lưu trong collection: '{FINAL_COLLECTION}'")
    print(f"\n💡 Cấu trúc document trong final_output:")
    print(f"   • title_clean: Tiêu đề đã làm sạch")
    print(f"   • content_clean: Nội dung đầy đủ đã làm sạch")
    print(f"   • summary: Tóm tắt bằng AI (ViT5)")
    print(f"   • analysis: Phân tích ngữ cảnh giao thông")
    print(f"   • keywords: Từ khóa chính")
    print(f"   • traffic_entities: Các thực thể giao thông được đề cập")
    print(f"   • category: Danh mục bài viết")
    
    print(f"\n{'='*70}\n")
    
    return stats

# ==================== CHẠY CHƯƠNG TRÌNH ====================

if __name__ == "__main__":
    import sys
    
    print(f"\n📝 Giai đoạn này sẽ:")
    print(f"   1. Load mô hình ViT5 đã fine-tune")
    print(f"   2. Tóm tắt từng bài báo trong clean_data")
    print(f"   3. Phân tích ngữ cảnh giao thông")
    print(f"   4. Lưu kết quả vào final_output collection")
    print(f"\n⏱️  Thời gian dự kiến: 5-30 phút tùy số lượng bài\n")
    
    # Chạy generation
    results = run_generation()
    
    if results:
        print(f"\n✅ Tóm tắt hoàn tất!")
        print(f"📊 Đã xử lý: {results['summarized']} bài")
        print(f"\n💡 Để xem kết quả, bạn có thể:")
        print(f"   • Kết nối MongoDB và query collection '{FINAL_COLLECTION}'")
        print(f"   • Chạy API server để truy vấn qua HTTP")

# HỆ THỐNG TÓM TẮT BÁO GIAO THÔNG TP.HCM 2025

## 📋 Tổng quan

Hệ thống tự động thu thập, làm sạch, và tóm tắt tin tức giao thông TP.HCM năm 2025 sử dụng mô hình **ViT5** (Vietnamese T5).

### 🎯 Mục tiêu
- Thu thập dữ liệu từ nhiều nguồn báo tiếng Việt
- Lọc chỉ giữ bài về giao thông TP.HCM năm 2025
- Làm sạch và chuẩn hóa dữ liệu
- Fine-tune mô hình ViT5 cho tóm tắt tiếng Việt
- Tạo summary và phân tích ngữ cảnh giao thông

---

## 🏗️ Kiến trúc hệ thống

```
┌─────────────────────────────────────────────────────────────┐
│ 1. DATA COLLECTION (1_data_collection.py)                  │
├─────────────────────────────────────────────────────────────┤
│ • Scrape từ RSS feeds + pagination                          │
│ • Lưu vào MongoDB: raw_data (dữ liệu thô)                  │
└─────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────┐
│ 2. DATA FILTERING (2_data_filtering.py)                    │
├─────────────────────────────────────────────────────────────┤
│ • Lọc: chỉ giữ bài giao thông TP.HCM 2025                  │
│ • Gán nhãn category tự động                                 │
│ • Lưu vào MongoDB: filtered_data                            │
└─────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────┐
│ 3. DATA CLEANING (3_data_cleaning.py)                      │
├─────────────────────────────────────────────────────────────┤
│ • Loại HTML, ký tự đặc biệt, spam                          │
│ • Chuẩn hóa unicode, dấu câu, khoảng trắng                 │
│ • Tách câu (sentence segmentation)                          │
│ • Lưu vào MongoDB: clean_data                               │
└─────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────┐
│ 4. MODEL TRAINING (4_model_training.py)                    │
├─────────────────────────────────────────────────────────────┤
│ • Load ViT5 pre-trained                                     │
│ • Fine-tune trên data giao thông TP.HCM                    │
│ • Split: 70% train, 15% val, 15% test                      │
│ • Lưu model: ./models/vit5_hcmc_traffic                    │
└─────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────┐
│ 5. MODEL EVALUATION (5_model_evaluation.py)                │
├─────────────────────────────────────────────────────────────┤
│ • Đánh giá trên test set                                    │
│ • Metrics: ROUGE-1, ROUGE-2, ROUGE-L, BLEU                 │
│ • Kiểm tra overfitting                                      │
│ • Lưu báo cáo: evaluation_results.json                      │
└─────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────┐
│ 6. GENERATE SUMMARIES (6_generate_summaries.py)            │
├─────────────────────────────────────────────────────────────┤
│ • Tóm tắt toàn bộ clean_data bằng ViT5                     │
│ • Phân tích ngữ cảnh giao thông                            │
│ • Trích xuất keywords                                       │
│ • Lưu vào MongoDB: final_output (KẾT QUẢ CUỐI CÙNG)       │
└─────────────────────────────────────────────────────────────┘
```

---

## 🔧 Cài đặt

### 1. Yêu cầu hệ thống
- **Python**: 3.8 hoặc mới hơn
- **MongoDB**: 4.0 hoặc mới hơn
- **GPU**: Khuyến nghị có GPU với 8GB+ VRAM (training sẽ chậm nếu dùng CPU)
- **RAM**: Tối thiểu 8GB
- **Disk**: Ít nhất 5GB trống

### 2. Cài đặt dependencies

```bash
# Tạo virtual environment (khuyến nghị)
python -m venv venv
source venv/bin/activate  # Linux/Mac
# hoặc
venv\Scripts\activate  # Windows

# Cài đặt packages
pip install -r requirements.txt
```

**requirements.txt:**
```
# Web scraping
selenium==4.15.0
beautifulsoup4==4.12.0
feedparser==6.0.10

# MongoDB
pymongo==4.6.0

# Machine Learning
torch==2.1.0
transformers==4.35.0
datasets==2.15.0

# Evaluation
rouge-score==0.1.2
nltk==3.8.1

# Utilities
numpy==1.24.0
scikit-learn==1.3.0

# Visualization
matplotlib==3.8.0

# API (optional)
fastapi==0.104.0
uvicorn==0.24.0
```

### 3. Cài đặt Edge WebDriver

```bash
# Download Edge WebDriver tương ứng với phiên bản Edge của bạn
# Đặt vào PATH hoặc cùng thư mục với code
```

### 4. Khởi động MongoDB

```bash
# Linux/Mac
sudo systemctl start mongod

# Windows: MongoDB sẽ tự chạy như service
# Hoặc chạy thủ công:
mongod --dbpath C:\data\db
```

---

## 🚀 Hướng dẫn sử dụng

### Quy trình đầy đủ (6 bước)

#### **Bước 1: Thu thập dữ liệu thô**
```bash
python 1_data_collection.py 50
# 50 = số bài tối đa mỗi nguồn
```
**Output:** MongoDB collection `raw_data`

---

#### **Bước 2: Lọc và gán nhãn**
```bash
python 2_data_filtering.py
```
**Output:** MongoDB collection `filtered_data` (chỉ bài về giao thông TP.HCM 2025)

---

#### **Bước 3: Làm sạch dữ liệu**
```bash
python 3_data_cleaning.py
```
**Output:** MongoDB collection `clean_data`

---

#### **Bước 4: Huấn luyện mô hình** ⚠️ **CẦN GPU**
```bash
python 4_model_training.py --auto
# Bỏ --auto nếu muốn xác nhận trước khi train
```
**Output:** 
- Model: `./models/vit5_hcmc_traffic/`
- Test set: `./models/vit5_hcmc_traffic/test_data.json`

⏱️ **Thời gian:** 1-3 giờ tùy GPU và số lượng dữ liệu

---

#### **Bước 5: Đánh giá mô hình**
```bash
python 5_model_evaluation.py
```
**Output:**
- `./models/vit5_hcmc_traffic/evaluation_results.json`
- `./models/vit5_hcmc_traffic/evaluation_examples.json`
- `./models/vit5_hcmc_traffic/training_curves.png`

---

#### **Bước 6: Tạo tóm tắt cho toàn bộ dữ liệu** 🎉
```bash
python 6_generate_summaries.py
```
**Output:** MongoDB collection `final_output` với:
- `summary`: Tóm tắt bằng ViT5
- `analysis`: Phân tích ngữ cảnh giao thông
- `keywords`: Từ khóa chính
- `traffic_entities`: Các thực thể giao thông

⏱️ **Thời gian:** 5-30 phút

---

## 📊 Cấu trúc MongoDB Collections

### 1. **raw_data** (Dữ liệu thô)
```json
{
  "url": "https://...",
  "source": "laodong",
  "title": "Tiêu đề gốc...",
  "content_text": "Nội dung gốc...",
  "content_html": "<html>...",
  "date_raw": "11/10/2025 14:30",
  "content_hash": "abc123...",
  "scraped_at": "2025-10-11T14:30:00",
  "processing_status": "raw"
}
```

### 2. **filtered_data** (Đã lọc)
```json
{
  "url": "https://...",
  "title": "Tiêu đề...",
  "content_text": "Nội dung...",
  "published_date": "2025-10-11T00:00:00",
  "category": "projects",
  "category_name": "Dự án giao thông",
  "is_hcmc": true,
  "is_traffic": true,
  "processing_status": "filtered"
}
```

### 3. **clean_data** (Đã làm sạch)
```json
{
  "url": "https://...",
  "title_clean": "Tiêu đề đã làm sạch",
  "content_clean": "Nội dung đã làm sạch...",
  "sentences": ["Câu 1.", "Câu 2.", ...],
  "num_sentences": 25,
  "num_words": 450,
  "category": "projects",
  "published_date": "2025-10-11T00:00:00",
  "processing_status": "cleaned"
}
```

### 4. **final_output** (KẾT QUẢ CUỐI CÙNG) ⭐
```json
{
  "url": "https://...",
  "title_clean": "Tiêu đề",
  "content_clean": "Nội dung đầy đủ...",
  
  "summary": "Tóm tắt bằng ViT5...",
  "analysis": "Chủ đề: Dự án giao thông. Hạ tầng: metro...",
  
  "keywords": ["metro", "thủ đức", "dự án", ...],
  "traffic_entities": {
    "mentioned_infrastructure": ["metro", "cầu"],
    "mentioned_locations": ["thủ đức", "quận 2"],
    "mentioned_issues": [],
    "mentioned_projects": ["dự án", "khởi công"],
    "main_topic": "Dự án giao thông"
  },
  
  "category": "projects",
  "published_date": "2025-10-11T00:00:00",
  "summary_length": 45,
  "summarized_at": "2025-10-12T10:00:00",
  "processing_status": "completed"
}
```

---

## 📈 Đánh giá mô hình

### Metrics sử dụng

| Metric | Ý nghĩa | Điểm tốt |
|--------|---------|----------|
| **ROUGE-1** | Độ trùng 1-gram (từ đơn) | > 0.35 |
| **ROUGE-2** | Độ trùng 2-gram (cụm 2 từ) | > 0.15 |
| **ROUGE-L** | Chuỗi con chung dài nhất | > 0.30 |
| **BLEU** | Độ chính xác n-gram | > 0.20 |

### Đánh giá Overfitting

- **Good**: Train/Val gap < 10%
- **Moderate**: Gap 10-20%
- **High**: Gap > 20% → Cần giảm model complexity hoặc tăng data

---

## ⚠️ Lưu ý quan trọng

### 1. **Về dữ liệu huấn luyện**
- Hiện tại code sử dụng **pseudo-labels**: lấy 3 câu đầu tiên làm summary
- Đây là phương pháp tạm thời, chưa tối ưu
- **Khuyến nghị**: Có human-labeled summaries để train tốt hơn

### 2. **Về GPU**
- Training ViT5 **yêu cầu GPU** (ít nhất 8GB VRAM)
- Nếu dùng CPU: rất chậm (10-20x)
- Giải pháp:
  - Dùng Google Colab (free GPU)
  - Giảm `BATCH_SIZE` trong code
  - Dùng mô hình nhỏ hơn (ViT5-small thay vì ViT5-base)

### 3. **Về số lượng dữ liệu**
- **Tối thiểu**: 100 bài để train
- **Khuyến nghị**: 500-1000 bài
- **Tốt nhất**: 2000+ bài

### 4. **Về thời gian**
| Bước | Thời gian | Yêu cầu |
|------|-----------|---------|
| 1. Collection | 10-30 phút | Internet |
| 2. Filtering | 1-5 phút | CPU |
| 3. Cleaning | 1-5 phút | CPU |
| 4. Training | 1-3 giờ | **GPU** |
| 5. Evaluation | 5-15 phút | GPU/CPU |
| 6. Generation | 5-30 phút | GPU/CPU |

---

## 🐛 Xử lý lỗi thường gặp

### Lỗi 1: "Can't connect to MongoDB"
```bash
# Kiểm tra MongoDB đã chạy chưa
sudo systemctl status mongod

# Khởi động MongoDB
sudo systemctl start mongod
```

### Lỗi 2: "Model not found"
→ Bạn chưa chạy bước 4 (training). Chạy `python 4_model_training.py` trước.

### Lỗi 3: "CUDA out of memory"
→ Giảm `BATCH_SIZE` trong file training (ví dụ: từ 8 → 4 → 2)

### Lỗi 4: "No data in collection"
→ Kiểm tra lại các bước trước đã chạy thành công chưa.

---

## 📚 Tài liệu tham khảo

- **ViT5 Model**: https://huggingface.co/VietAI/vit5-base
- **PhoBERT**: https://github.com/VinAIResearch/PhoBERT
- **Transformers**: https://huggingface.co/docs/transformers
- **ROUGE Metric**: https://github.com/google-research/google-research/tree/master/rouge

---

## 🤝 Đóng góp

Nếu bạn muốn cải thiện hệ thống:
1. Thêm nguồn tin mới vào `NEWS_SOURCES`
2. Cải thiện thuật toán phân loại
3. Thêm metrics đánh giá khác (BERTScore, METEOR)
4. Fine-tune với human-labeled data

---

## 📝 License

MIT License - Tự do sử dụng cho mục đích học tập và nghiên cứu.

---

## 📧 Liên hệ

Nếu có thắc mắc, vui lòng tạo issue trên GitHub hoặc liên hệ qua email.

---

**🎉 Chúc bạn thành công với đề tài tóm tắt báo giao thông TP.HCM 2025!**

API ĐỂ TRUY VẤN KẾT QUẢ TÓM TẮT
================================
FastAPI server để query dữ liệu từ final_output collection

In [ ]:
"""
API ĐỂ TRUY VẤN KẾT QUẢ TÓM TẮT
================================
FastAPI server để query dữ liệu từ final_output collection
"""

from fastapi import FastAPI, HTTPException, Query
from fastapi.middleware.cors import CORSMiddleware
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime
from typing import Optional, List
import json

# ==================== CẤU HÌNH ====================

MONGODB_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "hcmc_traffic_summarization"
FINAL_COLLECTION = "final_output"

# ==================== KẾT NỐI DATABASE ====================

try:
    client = MongoClient(MONGODB_URI, serverSelectionTimeoutMS=5000)
    client.server_info()
    db = client[DATABASE_NAME]
    final_col = db[FINAL_COLLECTION]
    print(f"✅ Đã kết nối MongoDB: {FINAL_COLLECTION}")
    print(f"   Tổng số bài: {final_col.count_documents({})}")
except Exception as e:
    print(f"❌ Lỗi MongoDB: {e}")
    final_col = None

# ==================== FASTAPI APP ====================

app = FastAPI(
    title="TP.HCM Traffic News Summarization API",
    description="API để truy vấn tin tức giao thông TP.HCM đã được tóm tắt bằng ViT5",
    version="1.0.0"
)

# CORS middleware (cho phép truy cập từ frontend)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ==================== HELPER FUNCTIONS ====================

def serialize_doc(doc):
    """
    Chuyển MongoDB document sang JSON-serializable dict
    """
    if doc is None:
        return None
    
    doc["id"] = str(doc["_id"])
    del doc["_id"]
    
    # Convert datetime to ISO string
    for key in ["published_date", "scraped_at", "cleaned_at", "summarized_at"]:
        if key in doc and isinstance(doc[key], datetime):
            doc[key] = doc[key].isoformat()
    
    return doc

# ==================== API ENDPOINTS ====================

@app.get("/")
def root():
    """
    Thông tin API
    """
    if final_col is None:
        raise HTTPException(status_code=503, detail="Database không khả dụng")
    
    total = final_col.count_documents({})
    
    return {
        "name": "TP.HCM Traffic News Summarization API",
        "version": "1.0.0",
        "description": "API để truy vấn tin tức giao thông TP.HCM đã tóm tắt",
        "total_articles": total,
        "endpoints": {
            "GET /": "API info",
            "GET /articles": "Danh sách bài viết (có filter)",
            "GET /articles/{id}": "Chi tiết 1 bài viết",
            "GET /search": "Tìm kiếm full-text",
            "GET /stats": "Thống kê tổng quan",
            "GET /categories": "Danh sách categories",
            "GET /trending": "Từ khóa trending"
        }
    }

@app.get("/articles")
def list_articles(
    category: Optional[str] = None,
    source: Optional[str] = None,
    from_date: Optional[str] = Query(None, description="YYYY-MM-DD"),
    to_date: Optional[str] = Query(None, description="YYYY-MM-DD"),
    limit: int = Query(20, le=100),
    skip: int = Query(0, ge=0)
):
    """
    Lấy danh sách bài viết với filter
    
    - **category**: Lọc theo danh mục (projects/infrastructure/issues/planning/transport)
    - **source**: Lọc theo nguồn (laodong/vnexpress/tuoitre)
    - **from_date**: Lọc từ ngày (YYYY-MM-DD)
    - **to_date**: Lọc đến ngày (YYYY-MM-DD)
    - **limit**: Số lượng kết quả (tối đa 100)
    - **skip**: Bỏ qua N kết quả đầu (cho pagination)
    """
    if final_col is None:
        raise HTTPException(status_code=503, detail="Database không khả dụng")
    
    # Build query
    query = {"processing_status": "completed"}
    
    if category:
        query["category"] = category
    
    if source:
        query["source"] = source
    
    # Date filter
    date_query = {}
    if from_date:
        try:
            date_query["$gte"] = datetime.strptime(from_date, "%Y-%m-%d")
        except:
            raise HTTPException(status_code=400, detail="Invalid from_date format")
    
    if to_date:
        try:
            date_query["$lte"] = datetime.strptime(to_date, "%Y-%m-%d")
        except:
            raise HTTPException(status_code=400, detail="Invalid to_date format")
    
    if date_query:
        query["published_date"] = date_query
    
    # Execute query
    cursor = final_col.find(query).sort("published_date", -1).skip(skip).limit(limit)
    
    articles = []
    for doc in cursor:
        articles.append({
            "id": str(doc["_id"]),
            "title": doc.get("title_clean", ""),
            "summary": doc.get("summary", ""),
            "analysis": doc.get("analysis", ""),
            "category": doc.get("category", ""),
            "category_name": doc.get("category_name", ""),
            "source": doc.get("source_name", ""),
            "published_date": doc.get("published_date", datetime.now()).isoformat(),
            "keywords": doc.get("keywords", [])[:5],  # Chỉ lấy 5 keywords
            "url": doc.get("url", "")
        })
    
    total = final_col.count_documents(query)
    
    return {
        "articles": articles,
        "count": len(articles),
        "total": total,
        "skip": skip,
        "limit": limit,
        "has_more": (skip + len(articles)) < total
    }

@app.get("/articles/{article_id}")
def get_article(article_id: str):
    """
    Lấy chi tiết đầy đủ của 1 bài viết
    """
    if final_col is None:
        raise HTTPException(status_code=503, detail="Database không khả dụng")
    
    try:
        doc = final_col.find_one({"_id": ObjectId(article_id)})
    except:
        raise HTTPException(status_code=400, detail="Invalid article ID")
    
    if doc is None:
        raise HTTPException(status_code=404, detail="Article not found")
    
    return serialize_doc(doc)

@app.get("/search")
def search_articles(
    q: str = Query(..., min_length=2, description="Từ khóa tìm kiếm"),
    limit: int = Query(20, le=100)
):
    """
    Tìm kiếm full-text trong title, summary, analysis
    """
    if final_col is None:
        raise HTTPException(status_code=503, detail="Database không khả dụng")
    
    # Full-text search
    cursor = final_col.find(
        {"$text": {"$search": q}},
        {"score": {"$meta": "textScore"}}
    ).sort([("score", {"$meta": "textScore"})]).limit(limit)
    
    results = []
    for doc in cursor:
        results.append({
            "id": str(doc["_id"]),
            "title": doc.get("title_clean", ""),
            "summary": doc.get("summary", ""),
            "category": doc.get("category", ""),
            "source": doc.get("source_name", ""),
            "published_date": doc.get("published_date", datetime.now()).isoformat(),
            "relevance_score": doc.get("score", 0),
            "url": doc.get("url", "")
        })
    
    return {
        "query": q,
        "results": results,
        "count": len(results)
    }

@app.get("/stats")
def get_statistics():
    """
    Thống kê tổng quan
    """
    if final_col is None:
        raise HTTPException(status_code=503, detail="Database không khả dụng")
    
    total = final_col.count_documents({"processing_status": "completed"})
    
    # By category
    pipeline_category = [
        {"$match": {"processing_status": "completed"}},
        {"$group": {"_id": "$category", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
    by_category = {
        doc["_id"]: doc["count"] 
        for doc in final_col.aggregate(pipeline_category)
    }
    
    # By source
    pipeline_source = [
        {"$match": {"processing_status": "completed"}},
        {"$group": {"_id": "$source_name", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
    by_source = {
        doc["_id"]: doc["count"] 
        for doc in final_col.aggregate(pipeline_source)
    }
    
    # By month
    pipeline_monthly = [
        {"$match": {"processing_status": "completed"}},
        {"$group": {
            "_id": {
                "year": {"$year": "$published_date"},
                "month": {"$month": "$published_date"}
            },
            "count": {"$sum": 1}
        }},
        {"$sort": {"_id.year": 1, "_id.month": 1}}
    ]
    monthly_data = []
    for doc in final_col.aggregate(pipeline_monthly):
        monthly_data.append({
            "month": f"{doc['_id']['year']}-{doc['_id']['month']:02d}",
            "count": doc["count"]
        })
    
    # Average summary length
    pipeline_avg = [
        {"$match": {"processing_status": "completed"}},
        {"$group": {
            "_id": None,
            "avg_summary_length": {"$avg": "$summary_length"},
            "avg_num_sentences": {"$avg": "$num_sentences"},
            "avg_num_words": {"$avg": "$num_words"}
        }}
    ]
    avg_stats = list(final_col.aggregate(pipeline_avg))
    avg_data = avg_stats[0] if avg_stats else {}
    
    return {
        "total_articles": total,
        "by_category": by_category,
        "by_source": by_source,
        "monthly_trend": monthly_data,
        "averages": {
            "summary_length": round(avg_data.get("avg_summary_length", 0), 1),
            "sentences_per_article": round(avg_data.get("avg_num_sentences", 0), 1),
            "words_per_article": round(avg_data.get("avg_num_words", 0), 1)
        }
    }

@app.get("/categories")
def list_categories():
    """
    Danh sách các categories và số lượng bài
    """
    if final_col is None:
        raise HTTPException(status_code=503, detail="Database không khả dụng")
    
    categories = [
        {
            "key": "projects",
            "name": "Dự án giao thông",
            "count": final_col.count_documents({"category": "projects"})
        },
        {
            "key": "infrastructure",
            "name": "Hạ tầng giao thông",
            "count": final_col.count_documents({"category": "infrastructure"})
        },
        {
            "key": "issues",
            "name": "Vấn đề giao thông",
            "count": final_col.count_documents({"category": "issues"})
        },
        {
            "key": "planning",
            "name": "Quy hoạch giao thông",
            "count": final_col.count_documents({"category": "planning"})
        },
        {
            "key": "transport",
            "name": "Phương tiện vận tải",
            "count": final_col.count_documents({"category": "transport"})
        },
        {
            "key": "general",
            "name": "Chung",
            "count": final_col.count_documents({"category": "general"})
        }
    ]
    
    return {
        "categories": categories,
        "total": sum(c["count"] for c in categories)
    }

@app.get("/trending")
def get_trending_keywords(top_n: int = Query(20, le=50)):
    """
    Top keywords trending (xuất hiện nhiều nhất)
    """
    if final_col is None:
        raise HTTPException(status_code=503, detail="Database không khả dụng")
    
    # Aggregate keywords
    pipeline = [
        {"$match": {"processing_status": "completed"}},
        {"$unwind": "$keywords"},
        {"$group": {"_id": "$keywords", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": top_n}
    ]
    
    trending = []
    for doc in final_col.aggregate(pipeline):
        trending.append({
            "keyword": doc["_id"],
            "count": doc["count"]
        })
    
    return {
        "trending_keywords": trending,
        "count": len(trending)
    }

@app.get("/sources")
def list_sources():
    """
    Danh sách các nguồn tin
    """
    if final_col is None:
        raise HTTPException(status_code=503, detail="Database không khả dụng")
    
    pipeline = [
        {"$match": {"processing_status": "completed"}},
        {"$group": {
            "_id": {
                "source": "$source",
                "source_name": "$source_name"
            },
            "count": {"$sum": 1}
        }},
        {"$sort": {"count": -1}}
    ]
    
    sources = []
    for doc in final_col.aggregate(pipeline):
        sources.append({
            "key": doc["_id"]["source"],
            "name": doc["_id"]["source_name"],
            "count": doc["count"]
        })
    
    return {
        "sources": sources,
        "total": sum(s["count"] for s in sources)
    }

@app.get("/export")
def export_data(
    format: str = Query("json", regex="^(json|csv)$"),
    category: Optional[str] = None,
    limit: int = Query(1000, le=10000)
):
    """
    Export dữ liệu ra JSON hoặc CSV
    """
    if final_col is None:
        raise HTTPException(status_code=503, detail="Database không khả dụng")
    
    query = {"processing_status": "completed"}
    if category:
        query["category"] = category
    
    cursor = final_col.find(query).limit(limit)
    
    articles = []
    for doc in cursor:
        articles.append({
            "id": str(doc["_id"]),
            "title": doc.get("title_clean", ""),
            "summary": doc.get("summary", ""),
            "analysis": doc.get("analysis", ""),
            "category": doc.get("category", ""),
            "source": doc.get("source_name", ""),
            "published_date": doc.get("published_date", datetime.now()).isoformat(),
            "url": doc.get("url", "")
        })
    
    if format == "csv":
        # Simple CSV
        csv_lines = ["ID,Title,Summary,Category,Source,Date,URL"]
        for art in articles:
            csv_lines.append(
                f"{art['id']},"
                f"\"{art['title']}\","
                f"\"{art['summary']}\","
                f"{art['category']},"
                f"{art['source']},"
                f"{art['published_date']},"
                f"{art['url']}"
            )
        
        return {
            "format": "csv",
            "data": "\n".join(csv_lines),
            "count": len(articles)
        }
    
    return {
        "format": "json",
        "articles": articles,
        "count": len(articles)
    }

# ==================== CHẠY SERVER ====================

if __name__ == "__main__":
    import uvicorn
    
    print(f"\n{'='*70}")
    print(f"🚀 KHỞI ĐỘNG API SERVER")
    print(f"{'='*70}")
    print(f"\n📡 Server sẽ chạy tại: http://localhost:8000")
    print(f"\n📖 Swagger UI (API docs): http://localhost:8000/docs")
    print(f"📖 ReDoc: http://localhost:8000/redoc")
    print(f"\n💡 Ví dụ sử dụng:")
    print(f"   curl http://localhost:8000/stats")
    print(f"   curl http://localhost:8000/articles?category=projects&limit=10")
    print(f"   curl http://localhost:8000/search?q=metro")
    print(f"\n{'='*70}\n")
    
    uvicorn.run(app, host="0.0.0.0", port=8000)